# Entity extraction: gpt-4.1 vs gpt-5.1

This notebook compares entity extraction quality across 50 English sentences using `extractEntitiesBatch` from `src/entities.ts`.

| Metric | gpt-5.1 (none) | gpt-5-mini (low) | gpt-5-nano (low) | gpt-4.1 | gpt-4.1 mini | gpt-4.1 nano |
| --- | --- | --- | --- | --- |--- |--- |
| Input     | 1.25 | 0.25 | 0.05 | 2.00 | 0.40 | 0.10 |
| Cached    | 0.13 | 0.03 | 0.01 | 0.50 | 0.10 | 0.03 |
| Output    | 10.00 | 2.00 | 0.40 | 8.00 | 1.60 | 0.4 |
| Reasoning/Intelligence | 4 | 3 | 2 | 4 | 3 | 2 |
| Speed     | 4 | 4 | 5 | 3 | 4 | 5 |
| Duration  | 2.7 | 8.3 | 9.0 | 4.2 | 2.5 | |


In [1]:
import { load } from "@std/dotenv";

import { extractEntitiesBatch } from "../src/entities.ts";

const { md, html } = Deno.jupyter;

In [2]:
void await load({
  envPath: "../.env",
  export: true,
});


## Entity definitions

We define a small set of generic entity types that both models will use when extracting entities.


In [3]:
const entityDefinitions: Record<string, string> = {
  person: "Individual people, including their roles or titles.",
  organization: "Companies, institutions, agencies, non-profits, and other named groups.",
  location: "Countries, cities, regions, landmarks, and other physical places.",
  brand: "Commercial brands, product lines, or trade names.",
  product: "Physical or digital products, services, or named offerings.",
  event: "Named events such as conferences, festivals, games, launches, or meetings.",
  date: "Specific calendar dates, months, or years mentioned in the text.",
  monetary_amount: "Any explicit amount of money or currency values.",
  percentage: "Any percentage or relative numeric share.",
};

const brandDefinitions: Record<string, string> = {
  brand: "Commercial brands, product lines, or trade names.",
  product: "Physical or digital products, services, or named offerings.",
};


## Test sentences

Fifty English sentences covering a mix of people, organizations, locations, products, events, dates, and numeric values.


In [4]:
const general_sentences: Array<string | null> = [
    "Apple announced the new iPhone 16 Pro in California yesterday.",
    "Google opened a new AI research center in Toronto last month.",
    "Elon Musk met with executives from Toyota in Tokyo on Tuesday.",
    "The conference on climate change will be held in Paris in November 2025.",
    "Amazon Web Services reduced prices for its S3 storage service by 10 percent.",
    "Microsoft acquired a cybersecurity startup based in Berlin for $200 million.",
    "The World Health Organization released a report about malaria in Sub-Saharan Africa.",
    "Tesla delivered its first Cybertruck vehicles to customers in Texas.",
    "Netflix plans to film a new documentary series in South Korea next year.",
    "The mayor of New York City announced a new public transport initiative.",
    "Harvard University partnered with MIT to launch an online education platform.",
    "The European Central Bank increased interest rates by 0.25 percentage points.",
    "Adobe released a major update to Photoshop for Windows and macOS users.",
    "The startup raised €5 million in seed funding from local investors.",
    "Uber is testing autonomous vehicles on the streets of San Francisco.",
    "The Olympic Games in Los Angeles are expected to attract millions of visitors.",
    "Spotify signed an exclusive podcast deal with a famous comedian.",
    "The Prime Minister of Canada visited Washington, D.C. for trade talks.",
    "Samsung unveiled a foldable smartphone at a technology expo in Barcelona.",
    "The local government in Madrid announced subsidies for solar panel installations.",
    "The charity organized a fundraising event for refugees in Jordan.",
    "Meta Platforms opened a new data center in Sweden powered by renewable energy.",
    "The CEO of Nestlé promised to reduce plastic packaging by 2030.",
    "Bitcoin briefly surpassed $80,000 during volatile trading on Monday.",
    "The museum in Amsterdam hosted a special exhibition of Van Gogh paintings.",
    "SpaceX successfully launched a Falcon 9 rocket from Cape Canaveral.",
    "The restaurant in downtown Chicago earned a second Michelin star.",
    "The football match between Real Madrid and Barcelona sold out in minutes.",
    "The startup accelerator in London accepted fifteen new fintech companies.",
    "The United Nations held an emergency meeting about the crisis in Yemen.",
    "The jazz festival in New Orleans features artists from Brazil and Japan.",
    "The hospital in Melbourne introduced a new robotic surgery system.",
    "The airline cancelled all flights from Heathrow because of heavy snow.",
    "The fashion brand opened a flagship store on Oxford Street in London.",
    "The university in Singapore launched a scholarship program for engineering students.",
    "The movie premiere at Cannes received a standing ovation from critics.",
    "The research team at Stanford discovered a new method for storing solar energy.",
    "The bank in Zurich introduced a mobile app for private wealth clients.",
    "The government of India announced a nationwide digital identity initiative.",
    "The mobile game became the top grossing app on the Apple App Store.",
    "The school in Dublin started a pilot program using tablets in classrooms.",
    "The biotech company in Boston began clinical trials for a new cancer drug.",
    "The regional train service between Munich and Vienna will be expanded next year.",
    "The art gallery in Lisbon showcased contemporary works by African artists.",
    "The insurance company offered discounted premiums for electric car owners.",
    "The manufacturing plant in Detroit invested in new robotics equipment.",
    "The embassy in Brussels organized a cultural event celebrating Japanese cuisine.",
    "The cloud computing startup moved its headquarters from Austin to Seattle.",
    "The hotel chain introduced a loyalty program for frequent business travelers.",
    "The logistics company opened a distribution center near the port of Rotterdam.",
];


In [5]:
interface BrandedSentence {
  text: string;
  brands: Array<string>;
  products: Array<string | Array<string>>;
}

const brand_sentences: Array<BrandedSentence> = [
  {
    text: "I just bought the new Sony PlayStation 5 and a Samsung QLED TV.",
    brands: ["Sony", "Samsung"],
    products: [["PlayStation 5", "Sony PlayStation 5", "PlayStation"], ["QLED TV", "Samsung QLED TV", "TV"]]
  },
  {
    text: "My new Nike running shoes are more comfortable than my old Adidas ones.",
    brands: ["Nike", "Adidas"],
    products: [["running shoes", "Nike running shoes", "shoes"]]
  },
  {
    text: "We ordered a Domino's pizza and some Coca-Cola for the movie night.",
    brands: ["Domino's", "Coca-Cola"],
    products: [["pizza", "Domino's pizza"], "Coca-Cola"]
  },
  {
    text: "She loves her new Apple iPhone 14 Pro, especially the camera.",
    brands: ["Apple"],
    products: [["iPhone 14 Pro", "Apple iPhone 14 Pro", "iPhone"]]
  },
  {
    text: "He is considering buying a Ford Mustang or a Chevrolet Camaro.",
    brands: ["Ford", "Chevrolet"],
    products: [["Mustang", "Ford Mustang"], ["Camaro", "Chevrolet Camaro"]]
  },
  {
    text: "For breakfast, I had Kellogg's Corn Flakes with Nestlé milk.",
    brands: ["Kellogg's", "Nestlé"],
    products: [["Corn Flakes", "Kellogg's Corn Flakes"], ["milk", "Nestlé milk"]]
  },
  {
    text: "I'm using a Dell XPS laptop with a Logitech mouse for work.",
    brands: ["Dell", "Logitech"],
    products: [["XPS laptop", "Dell XPS", "laptop"], ["mouse", "Logitech mouse"]]
  },
  {
    text: "The new Tesla Model Y has impressive range and performance.",
    brands: ["Tesla"],
    products: [["Model Y", "Tesla Model Y"]]
  },
  {
    text: "We use Microsoft Office 365 for all our documents and spreadsheets.",
    brands: ["Microsoft"],
    products: [["Office 365", "Microsoft Office 365", "Office"]]
  },
  {
    text: "I'm drinking a Starbucks Frappuccino while reading a book on my Amazon Kindle.",
    brands: ["Starbucks", "Amazon"],
    products: [["Frappuccino", "Starbucks Frappuccino"], ["Kindle", "Amazon Kindle"]]
  },
  {
    text: "My dentist recommended using a Philips Sonicare toothbrush.",
    brands: ["Philips"],
    products: [["Sonicare toothbrush", "Philips Sonicare toothbrush", "toothbrush"]]
  },
  {
    text: "The North Face jackets are great for cold weather, but Patagonia is more sustainable.",
    brands: ["The North Face", "Patagonia"],
    products: [["jackets", "The North Face jackets", "North Face jackets"]]
  },
  {
    text: "I'm listening to music on Spotify with my Bose QuietComfort headphones.",
    brands: ["Spotify", "Bose"],
    products: ["Spotify", ["QuietComfort headphones", "Bose QuietComfort headphones", "headphones"]]
  },
  {
    text: "We're planning a trip to Disneyland and staying at a Hilton hotel.",
    brands: ["Disneyland", "Hilton"],
    products: ["Disneyland", ["hotel", "Hilton hotel"]]
  },
  {
    text: "He bought a new Honda Civic, which is more fuel-efficient than his old Jeep.",
    brands: ["Honda", "Jeep"],
    products: [["Civic", "Honda Civic"], "Jeep"]
  },
  {
    text: "I need to buy some Pampers diapers and Johnson's baby powder.",
    brands: ["Pampers", "Johnson's"],
    products: [["diapers", "Pampers diapers"], ["baby powder", "Johnson's baby powder", "powder"]]
  },
  {
    text: "She's wearing a pair of Levi's jeans and a Gap t-shirt.",
    brands: ["Levi's", "Gap"],
    products: [["jeans", "Levi's jeans"], ["t-shirt", "Gap t-shirt"]]
  },
  {
    text: "I'm watching a Netflix series on my LG OLED TV.",
    brands: ["Netflix", "LG"],
    products: [["series", "Netflix series"], ["OLED TV", "LG OLED TV", "TV"]]
  },
  {
    text: "He prefers the taste of Pepsi over Coca-Cola.",
    brands: ["Pepsi", "Coca-Cola"],
    products: ["Pepsi", "Coca-Cola"]
  },
  {
    text: "My new Canon EOS R5 camera takes amazing photos.",
    brands: ["Canon"],
    products: [["EOS R5", "Canon EOS R5 camera", "EOS R5 camera", "camera", "R5", "R5 camera"]]
  },
  {
    text: "I'm using Google Chrome on my MacBook Pro.",
    brands: ["Google"],
    products: [["Chrome", "Google Chrome"], ["MacBook Pro", "MacBook"]]
  },
  {
    text: "She bought a Louis Vuitton handbag and a pair of Gucci shoes.",
    brands: ["Louis Vuitton", "Gucci"],
    products: [["handbag", "Louis Vuitton handbag"], ["shoes", "Gucci shoes"]]
  },
  {
    text: "We're flying with Emirates to Dubai and then with Qantas to Sydney.",
    brands: ["Emirates", "Qantas"],
    products: []
  },
  {
    text: "I'm using an NVIDIA GeForce RTX 4090 graphics card in my gaming PC.",
    brands: ["NVIDIA"],
    products: [["GeForce RTX 4090", "NVIDIA GeForce RTX 4090 graphics card", "graphics card", "RTX 4090"]]
  },
  {
    text: "He's eating a McDonald's Big Mac with a side of Heinz ketchup.",
    brands: ["McDonald's", "Heinz"],
    products: [["Big Mac", "McDonald's Big Mac"], ["ketchup", "Heinz ketchup"]]
  },
  {
    text: "I'm writing notes in a Moleskine notebook with a Pilot G2 pen.",
    brands: ["Moleskine", "Pilot"],
    products: [["notebook", "Moleskine notebook"], ["G2 pen", "Pilot G2 pen", "pen"]]
  },
  {
    text: "She's using a Dyson Supersonic hair dryer.",
    brands: ["Dyson"],
    products: [["Supersonic hair dryer", "Dyson Supersonic hair dryer", "hair dryer"]]
  },
  {
    text: "We're using Adobe Photoshop and Illustrator for our design project.",
    brands: ["Adobe"],
    products: [["Photoshop", "Adobe Photoshop"], ["Illustrator", "Adobe Illustrator"]]
  },
  {
    text: "I'm wearing a Rolex Submariner watch.",
    brands: ["Rolex"],
    products: [["Submariner watch", "Rolex Submariner watch"]]
  },
  {
    text: "He's playing Call of Duty on his Xbox Series X.",
    brands: ["Xbox"],
    products: ["Call of Duty", ["Xbox Series X", "Microsoft Xbox Series X", "Xbox"]]
  },
  {
    text: "I'm drinking a bottle of Evian water.",
    brands: ["Evian"],
    products: [["water", "Evian water"]]
  },
  {
    text: "She's using a KitchenAid mixer to bake a cake.",
    brands: ["KitchenAid"],
    products: [["mixer", "KitchenAid mixer"]]
  },
  {
    text: "We're shopping for groceries at Walmart and Whole Foods.",
    brands: ["Walmart", "Whole Foods"],
    products: []
  },
  {
    text: "I'm using a Weber grill for the barbecue this weekend.",
    brands: ["Weber"],
    products: [["grill", "Weber grill"]]
  },
  {
    text: "He's taking an Uber to the airport to catch a British Airways flight.",
    brands: ["Uber", "British Airways"],
    products: [["Uber", "Uber ride"], ["flight", "British Airways flight"]]
  },
  {
    text: "I'm using a Herman Miller Aeron chair in my home office.",
    brands: ["Herman Miller"],
    products: [["Aeron chair", "Herman Miller Aeron chair", "chair"]]
  },
  {
    text: "She's using a L'Oréal shampoo and a Dove soap.",
    brands: ["L'Oréal", "Dove"],
    products: [["shampoo", "L'Oréal shampoo"], ["soap", "Dove soap"]]
  },
  {
    text: "We're using Slack and Zoom for team communication.",
    brands: ["Slack", "Zoom"],
    products: ["Slack", "Zoom"]
  },
  {
    text: "I'm eating a bag of Lay's potato chips.",
    brands: ["Lay's"],
    products: [["potato chips", "Lay's potato chips", "chips"]]
  },
  {
    text: "He's driving a Toyota Prius, which is a popular hybrid car.",
    brands: ["Toyota"],
    products: [["Prius", "Toyota Prius"]]
  },
  {
    text: "I'm using a GoPro HERO11 to record my vacation.",
    brands: ["GoPro"],
    products: [["HERO11", "GoPro HERO11"]]
  },
  {
    text: "She's wearing a pair of Ray-Ban sunglasses.",
    brands: ["Ray-Ban"],
    products: [["sunglasses", "Ray-Ban sunglasses"]]
  },
  {
    text: "We're using a Roomba to clean the house.",
    brands: ["Roomba"],
    products: [["Roomba"]]
  },
  {
    text: "I'm using a Fitbit to track my steps and heart rate.",
    brands: ["Fitbit"],
    products: [["Fitbit"]]
  },
  {
    text: "He's drinking a Red Bull energy drink.",
    brands: ["Red Bull"],
    products: [["energy drink", "Red Bull energy drink"]]
  },
  {
    text: "I'm using a Yeti tumbler to keep my coffee hot.",
    brands: ["Yeti"],
    products: [["tumbler", "Yeti tumbler"]]
  },
  {
    text: "She's using a MAC lipstick and a Fenty Beauty foundation.",
    brands: ["MAC", "Fenty Beauty"],
    products: [["lipstick", "MAC lipstick"], ["foundation", "Fenty Beauty foundation"]]
  },
  {
    text: "We're using a Nest thermostat to control the temperature in our home.",
    brands: ["Nest"],
    products: [["thermostat", "Nest thermostat"]]
  },
  {
    text: "I'm eating a bowl of Ben & Jerry's ice cream.",
    brands: ["Ben & Jerry's"],
    products: [["ice cream", "Ben & Jerry's ice cream"]]
  },
  {
    text: "He's using a Fender Stratocaster guitar to play a song by The Beatles.",
    brands: ["Fender", "The Beatles"],
    products: [["Stratocaster guitar", "Fender Stratocaster guitar", "guitar", "stratocaster"]]
  }
];

## Run entity extraction for configured models

This cell iterates over every entry in the `models` array, runs `extractEntitiesBatch` with the provided configuration, and produces a side-by-side comparison.


In [6]:
interface ModelConfig {
    name: string;
    label?: string;
    options?: Record<string, unknown>;
}

const models: Array<ModelConfig> = [
    {
        name: "gpt-5.1",
        label: "GPT-5.1 (none)",
        options: { reasoning: { effort: "none" } }
    },
    {
        name: "gpt-5-mini",
        label: "GPT-5 Mini (low)",
        options: { reasoning: { effort: "low" } }
    },
    {
        name: "gpt-5-nano",
        label: "GPT-5 Nano (low)",
        options: { reasoning: { effort: "low" } }
    },
    { name: "gpt-4.1" },
    { name: "gpt-4.1-mini" },
    { name: "gpt-4.1-nano" },
];

In [7]:
let sentences = brand_sentences.map((s) => s.text);
let modelRuns: Array<unknown> = [];

In [8]:
const extraModels: Array<ModelConfig> = [
    { name: "gpt-4.1-nano" },
];

In [31]:
let ifExists: string = 'overwrite';
let execModels = extraModels;

for (const model of execModels) {
    const label = model.label ?? model.name;
    const exists = modelRuns.findIndex((run) => run.model === label);
    if (exists != -1) {
        if (ifExists === 'ignore') {
            console.log(`Skipping ${label}, already run.`);
            continue;
        }
    }
    console.log(`Running ${label}...`);
    const startedAt = performance.now();
    const entities = await extractEntitiesBatch(
        sentences,
        brandDefinitions,
        model.name,
        100,
        model.options
    );
    const durationMs = performance.now() - startedAt;
    const run = {
        model: label,
        entities,
        durationMs
    };
    if (exists != -1) {
        modelRuns[exists] = run;
    } else {
        modelRuns.push(run);
    }
}

Running gpt-4.1-nano...


{
  model: "gpt-4.1-nano",
  entities: [
    [
      { name: "sony", type: "brand" },
      { name: "playstation 5", type: "product" },
      { name: "samsung", type: "brand" },
      { name: "qled tv", type: "product" }
    ],
    [
      { name: "nike", type: "brand" },
      { name: "adidas", type: "brand" },
      { name: "running shoes", type: "product" }
    ],
    [
      { name: "domino's", type: "brand" },
      { name: "coca-cola", type: "brand" }
    ],
    [
      { name: "apple", type: "brand" },
      { name: "iphone 14", type: "product" }
    ],
    [
      { name: "ford", type: "brand" },
      { name: "mustang", type: "product" },
      { name: "chevrolet", type: "brand" },
      { name: "camaro", type: "product" }
    ],
    [
      { name: "kellogg's", type: "brand" },
      { name: "nestlé", type: "brand" }
    ],
    [
      { name: "dell", type: "brand" },
      { name: "logitech", type: "brand" },
      { name: "xps", type: "product" },
      { name: "laptop", ty

In [32]:
const comparisons = sentences.map((text, index) => ({
  index,
  text,
  entitiesByModel: modelRuns.map((run) => ({
    model: run.model,
    entities: run.entities[index] ?? []
  }))
}));

const processingTimes = modelRuns.map((run) => ({
  model: run.model,
  durationMs: run.durationMs
}));

processingTimes;

[
  { model: "GPT-5.1 (none)", durationMs: 6904.5365409999795 },
  { model: "GPT-5 Mini (low)", durationMs: 11109.592249999987 },
  { model: "GPT-5 Nano (low)", durationMs: 7527.064207999996 },
  { model: "gpt-4.1", durationMs: 6597.220624999987 },
  { model: "gpt-4.1-nano", durationMs: 4991.962999999989 },
  { model: "gpt-4.1-mini", durationMs: 9027.821457999991 }
]

In [33]:
comparisons[0]?.entitiesByModel.map((entry) => ({
    model: entry.model,
    names: entry.entities.map((entity) => entity.name),
    types: entry.entities.map((entity) => entity.type)
}));

[
  {
    model: "GPT-5.1 (none)",
    names: [ "sony", "playstation 5", "samsung", "qled tv" ],
    types: [ "brand", "product", "brand", "product" ]
  },
  {
    model: "GPT-5 Mini (low)",
    names: [ "sony", "playstation 5", "samsung", "qled tv" ],
    types: [ "brand", "product", "brand", "product" ]
  },
  {
    model: "GPT-5 Nano (low)",
    names: [ "sony", "playstation 5", "samsung", "qled tv" ],
    types: [ "brand", "product", "brand", "product" ]
  },
  {
    model: "gpt-4.1",
    names: [ "sony", "playstation 5", "samsung", "qled tv" ],
    types: [ "brand", "product", "brand", "product" ]
  },
  {
    model: "gpt-4.1-nano",
    names: [ "sony", "playstation 5", "samsung", "qled tv" ],
    types: [ "brand", "product", "brand", "product" ]
  },
  {
    model: "gpt-4.1-mini",
    names: [ "sony", "playstation 5", "samsung", "qled tv" ],
    types: [ "brand", "product", "brand", "product" ]
  }
]

In [36]:
function normalizeVariants(value: string): Array<string> {
    const trimmed = value.trim().toLowerCase();
    const baseCandidates = new Set<string>([
        trimmed,
        trimmed.normalize('NFD').replace(/[\u0300-\u036f]/g, '')
    ]);
    const variants = new Set<string>();
    for (const candidate of baseCandidates) {
        if (candidate.length === 0) {
            continue;

        }
        const noQuotes = candidate.replace(/['\u2019`]/g, '');
        const wordsOnly = candidate.replace(/[^a-z0-9]+/g, ' ').trim().replace(/\s+/g, ' ');
        const noSpaces = wordsOnly.replace(/\s+/g, '');
        variants.add(candidate);
        if (noQuotes.length > 0) {
            variants.add(noQuotes);

        }
        if (wordsOnly.length > 0) {
            variants.add(wordsOnly);

        }
        if (noSpaces.length > 0) {
            variants.add(noSpaces);

        }
        if (noQuotes.endsWith('s') && noQuotes.length > 1) {
            variants.add(noQuotes.slice(0, -1));

        }
        if (noSpaces.endsWith('s') && noSpaces.length > 1) {
            variants.add(noSpaces.slice(0, -1));

        }

    }
    return Array.from(variants);
}

interface ExtractedEntity {
    name: string;
    type: string;
}

interface GroundTruthEntry {
    type: 'brand' | 'product';
    alternatives: Array<string>;
}

const groundTruthBySentence: Array<Array<GroundTruthEntry>> = brand_sentences.map((sentence) => {
    const entries: Array<GroundTruthEntry> = [];
    for (const brand of sentence.brands) {
        entries.push({
            type: 'brand',
            alternatives: [brand]

        });

    }
    for (const product of sentence.products) {
        const alternatives: Array<string> = Array.isArray(product) ? product : [product];
        entries.push({
            type: 'product',
            alternatives

        });

    }
    return entries;
});

interface ModelRun {
    model: string;
    entities: Array<Array<ExtractedEntity>>;
    durationMs: number;
}

const typedModelRuns = modelRuns as Array<ModelRun>;

const accuracyByModel = typedModelRuns.map((run) => {
    let matched = 0;
    let total = 0;
    run.entities.forEach((entitiesForSentence, sentenceIndex) => {
        const groundTruthEntries = groundTruthBySentence[sentenceIndex] ?? [];
        const normalizedByType = (entitiesForSentence ?? []).reduce<Record<string, Set<string>>>((acc, entity) => {
            const variants = normalizeVariants(entity.name);
            const bucket = acc[entity.type] ?? new Set<string>();
            for (const variant of variants) {
                bucket.add(variant);

            }
            acc[entity.type] = bucket;
            return acc;

        }, {});
        for (const truth of groundTruthEntries) {
            total += 1;
            const candidates = normalizedByType[truth.type];
            if (!candidates) {
                continue;

            }
            const altVariants = new Set<string>();
            for (const alternative of truth.alternatives) {
                for (const variant of normalizeVariants(alternative)) {
                    altVariants.add(variant);

                }

            }
            for (const variant of altVariants) {
                if (candidates.has(variant)) {
                    matched += 1;
                    break;

                }

            }

        }
    });
    const accuracy = total === 0 ? 0 : matched / total;
    return {
        model: run.model,
        matched,
        total,
        accuracy

    };
});

const accuracyTableHeader = '| Model | Accuracy | Matched | Total |';
const accuracyTableSeparator = '| --- | --- | --- | --- |';
const accuracyTableRows = accuracyByModel.map((result) => {
    const percentage = `${(result.accuracy * 100).toFixed(1)}%`;
    return `| ${result.model} | ${percentage} | ${result.matched} | ${result.total} |`;
});

md`${[accuracyTableHeader, accuracyTableSeparator, ...accuracyTableRows].join('\n')}`;

| Model | Accuracy | Matched | Total |
| --- | --- | --- | --- |
| GPT-5.1 (none) | 87.7% | 128 | 146 |
| GPT-5 Mini (low) | 92.5% | 135 | 146 |
| GPT-5 Nano (low) | 89.7% | 131 | 146 |
| gpt-4.1 | 87.0% | 127 | 146 |
| gpt-4.1-nano | 71.9% | 105 | 146 |
| gpt-4.1-mini | 89.0% | 130 | 146 |

In [35]:
function escapeForMarkdown(text: string): string {
  return text.replace(/\|/g, "\\|").replace(/\n/g, "<br>");
}

const headerCells = [
  "#",
  "Sentence",
  ...modelRuns.flatMap((run) => [`${run.model} entities`])
];
const header = `| ${headerCells.join(" | ")} |`;
const separator = `| ${headerCells.map(() => "---").join(" | ")} |`;

const rows = comparisons.map((row) => {
  const cells = [
    String(row.index + 1),
    escapeForMarkdown(row.text ?? "")
  ];

  for (const entry of row.entitiesByModel) {
    // const names = escapeForMarkdown(entry.entities.sort().map((entity) => entity.name + " – " + entity.type).join("\n"));
    const names = escapeForMarkdown(
      entry.entities
        .sort((a, b) => a.name.localeCompare(b.name))
        .map((entity) => entity.name + " – " + entity.type)
        .join("\n")
    );
    cells.push(names);
  }

  return `| ${cells.join(" | ")} |`;
});

const markdownTable = [header, separator, ...rows].join("\n");

md`${markdownTable}`;

| # | Sentence | GPT-5.1 (none) entities | GPT-5 Mini (low) entities | GPT-5 Nano (low) entities | gpt-4.1 entities | gpt-4.1-nano entities | gpt-4.1-mini entities |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | I just bought the new Sony PlayStation 5 and a Samsung QLED TV. | playstation 5 – product<br>qled tv – product<br>samsung – brand<br>sony – brand | playstation 5 – product<br>qled tv – product<br>samsung – brand<br>sony – brand | playstation 5 – product<br>qled tv – product<br>samsung – brand<br>sony – brand | playstation 5 – product<br>qled tv – product<br>samsung – brand<br>sony – brand | playstation 5 – product<br>qled tv – product<br>samsung – brand<br>sony – brand | playstation 5 – product<br>qled tv – product<br>samsung – brand<br>sony – brand |
| 2 | My new Nike running shoes are more comfortable than my old Adidas ones. | adidas – brand<br>nike – brand<br>running shoe – product | adidas – brand<br>nike – brand<br>running shoe – product | adidas – brand<br>nike – brand<br>running shoe – product | adidas – brand<br>nike – brand<br>running shoe – product | adidas – brand<br>nike – brand<br>running shoes – product | adidas – brand<br>nike – brand<br>running shoe – product |
| 3 | We ordered a Domino's pizza and some Coca-Cola for the movie night. | coca-cola – brand<br>domino's – brand<br>pizza – product | coca-cola – brand<br>coca-cola – product<br>domino's – brand<br>pizza – product | coca-cola – brand<br>coca-cola – product<br>domino's – brand<br>pizza – product | coca-cola – brand<br>domino's – brand<br>pizza – product | coca-cola – brand<br>domino's – brand | coca-cola – brand<br>domino's – brand<br>pizza – product |
| 4 | She loves her new Apple iPhone 14 Pro, especially the camera. | apple – brand<br>iphone 14 pro – product | apple – brand<br>iphone 14 pro – product | apple – brand<br>iphone 14 pro – product | apple – brand<br>iphone 14 pro – product | apple – brand<br>iphone 14 – product | apple – brand<br>iphone 14 pro – product |
| 5 | He is considering buying a Ford Mustang or a Chevrolet Camaro. | camaro – product<br>chevrolet – brand<br>ford – brand<br>mustang – product | camaro – product<br>chevrolet – brand<br>ford – brand<br>mustang – product | camaro – product<br>chevrolet – brand<br>ford – brand<br>mustang – product | camaro – product<br>chevrolet – brand<br>ford – brand<br>mustang – product | camaro – product<br>chevrolet – brand<br>ford – brand<br>mustang – product | camaro – product<br>chevrolet – brand<br>ford – brand<br>mustang – product |
| 6 | For breakfast, I had Kellogg's Corn Flakes with Nestlé milk. | corn flakes – product<br>kellogg's – brand<br>milk – product<br>nestlé – brand | corn flake – product<br>kellogg's – brand<br>milk – product<br>nestlé – brand | corn flakes – product<br>kellogg's – brand<br>milk – product<br>nestlé – brand | corn flakes – product<br>kellogg's – brand<br>milk – product<br>nestlé – brand | kellogg's – brand<br>nestlé – brand | corn flakes – product<br>kellogg's – brand<br>milk – product<br>nestlé – brand |
| 7 | I'm using a Dell XPS laptop with a Logitech mouse for work. | dell – brand<br>laptop – product<br>logitech – brand<br>mouse – product<br>xps – product | dell – brand<br>logitech – brand<br>logitech mouse – product<br>xps laptop – product | dell – brand<br>logitech – brand<br>logitech mouse – product<br>xps laptop – product | dell – brand<br>logitech – brand<br>mouse – product<br>xps – product | dell – brand<br>laptop – product<br>logitech – brand<br>mouse – product<br>xps – product | dell – brand<br>logitech – brand<br>mouse – product<br>xps laptop – product |
| 8 | The new Tesla Model Y has impressive range and performance. | model y – product<br>tesla – brand | model y – product<br>tesla – brand | model y – product<br>tesla – brand | model y – product<br>tesla – brand | model y – product<br>tesla – brand | model y – product<br>tesla – brand |
| 9 | We use Microsoft Office 365 for all our documents and spreadsheets. | microsoft – brand<br>microsoft office 365 – product | microsoft – brand<br>office 365 – product | microsoft – brand<br>office 365 – product | microsoft – brand<br>office 365 – product | microsoft – brand<br>office 365 – product | microsoft – brand<br>office 365 – product |
| 10 | I'm drinking a Starbucks Frappuccino while reading a book on my Amazon Kindle. | amazon – brand<br>frappuccino – product<br>kindle – product<br>starbucks – brand | amazon – brand<br>frappuccino – product<br>kindle – product<br>starbucks – brand | amazon – brand<br>frappuccino – product<br>kindle – product<br>starbucks – brand | amazon – brand<br>frappuccino – product<br>kindle – product<br>starbucks – brand | kindle – product<br>starbucks – brand | amazon – brand<br>frappuccino – product<br>kindle – product<br>starbucks – brand |
| 11 | My dentist recommended using a Philips Sonicare toothbrush. | philips – brand<br>sonicare – brand<br>sonicare toothbrush – product<br>toothbrush – product | philips – brand<br>sonicare toothbrush – product | philips sonicare – brand<br>toothbrush – product | philips sonicare – brand<br>toothbrush – product | philips – brand<br>sonicare – product<br>toothbrush – product | philips sonicare – brand<br>toothbrush – product |
| 12 | The North Face jackets are great for cold weather, but Patagonia is more sustainable. | jacket – product<br>patagonia – brand<br>the north face – brand | jacket – product<br>patagonia – brand<br>the north face – brand | jacket – product<br>patagonia – brand<br>the north face – brand | jacket – product<br>patagonia – brand<br>the north face – brand | north face – brand<br>patagonia – brand | jacket – product<br>patagonia – brand<br>the north face – brand |
| 13 | I'm listening to music on Spotify with my Bose QuietComfort headphones. | bose – brand<br>quietcomfort – product<br>spotify – brand | bose – brand<br>quietcomfort headphone – product<br>spotify – brand | bose – brand<br>quietcomfort headphone – product<br>spotify – brand | bose – brand<br>quietcomfort – product<br>spotify – brand | bose quietcomfort – brand<br>spotify – brand | bose – brand<br>quietcomfort headphone – product<br>spotify – brand |
| 14 | We're planning a trip to Disneyland and staying at a Hilton hotel. | disneyland – brand<br>hilton – brand<br>hotel – product | disneyland – brand<br>hilton – brand | disneyland – brand<br>hilton hotel – product | disneyland – brand<br>hilton – brand | disneyland – brand<br>hilton – brand | disneyland – brand<br>hilton – brand |
| 15 | He bought a new Honda Civic, which is more fuel-efficient than his old Jeep. | civic – product<br>honda – brand<br>jeep – brand | honda – brand<br>honda civic – product<br>jeep – brand | civic – product<br>honda – brand<br>jeep – brand | civic – product<br>honda – brand<br>jeep – brand | civic – product<br>honda – brand<br>jeep – brand | civic – product<br>honda – brand<br>jeep – brand |
| 16 | I need to buy some Pampers diapers and Johnson's baby powder. | baby powder – product<br>diaper – product<br>johnson's – brand<br>pampers – brand | baby powder – product<br>diaper – product<br>johnson's – brand<br>pampers – brand | baby powder – product<br>diaper – product<br>johnson's – brand<br>pampers – brand | baby powder – product<br>diaper – product<br>johnson's – brand<br>pampers – brand | johnson's baby powder – product<br>pampers – brand | johnson's baby powder – product<br>pampers diaper – product |
| 17 | She's wearing a pair of Levi's jeans and a Gap t-shirt. | gap – brand<br>jeans – product<br>levi's – brand<br>t-shirt – product | gap – brand<br>jean – product<br>levi's – brand<br>t-shirt – product | gap – brand<br>jeans – product<br>levi's – brand<br>t-shirt – product | gap – brand<br>jean – product<br>levi's – brand<br>t-shirt – product | gap – brand<br>jeans – product<br>levi's – brand<br>t-shirt – product | gap – brand<br>jean – product<br>levi's – brand<br>t-shirt – product |
| 18 | I'm watching a Netflix series on my LG OLED TV. | lg – brand<br>netflix – brand<br>oled tv – product | lg – brand<br>lg oled tv – product<br>netflix – brand<br>netflix series – product | lg – brand<br>netflix – brand<br>oled tv – product | lg – brand<br>netflix – brand<br>oled tv – product | netflix – brand<br>oled tv – product | lg – brand<br>netflix – brand<br>oled tv – product |
| 19 | He prefers the taste of Pepsi over Coca-Cola. | coca-cola – brand<br>pepsi – brand | coca-cola – brand<br>pepsi – brand | coca-cola – brand<br>pepsi – brand | coca-cola – brand<br>pepsi – brand | coca-cola – brand<br>pepsi – brand | coca-cola – brand<br>pepsi – brand |
| 20 | My new Canon EOS R5 camera takes amazing photos. | camera – product<br>canon – brand<br>eos r5 – product | canon – brand<br>eos r5 camera – product | canon – brand<br>eos r5 camera – product | canon – brand<br>eos r5 – product | canon – brand<br>eos r5 – product | canon – brand<br>eos r5 camera – product |
| 21 | I'm using Google Chrome on my MacBook Pro. | apple – brand<br>google – brand<br>google chrome – product<br>macbook pro – product | google chrome – product<br>macbook pro – product | google – brand<br>google chrome – product<br>macbook pro – product | apple – brand<br>google – brand<br>google chrome – product<br>macbook pro – product | google chrome – brand | google chrome – product<br>macbook pro – product |
| 22 | She bought a Louis Vuitton handbag and a pair of Gucci shoes. | gucci – brand<br>handbag – product<br>louis vuitton – brand<br>shoes – product | gucci – brand<br>handbag – product<br>louis vuitton – brand<br>shoe – product | gucci – brand<br>handbag – product<br>louis vuitton – brand<br>shoe – product | gucci – brand<br>handbag – product<br>louis vuitton – brand<br>shoe – product | gucci – brand<br>louis vuitton – brand | gucci – brand<br>handbag – product<br>louis vuitton – brand<br>shoe – product |
| 23 | We're flying with Emirates to Dubai and then with Qantas to Sydney. | emirates – brand<br>qantas – brand | emirates – brand<br>flight – product<br>qantas – brand | emirates – brand<br>flight – product<br>qantas – brand | emirates – brand<br>qantas – brand | emirates – brand<br>qantas – brand | emirates – brand<br>qantas – brand |
| 24 | I'm using an NVIDIA GeForce RTX 4090 graphics card in my gaming PC. | gaming pc – product<br>geforce rtx 4090 – product<br>graphics card – product<br>nvidia – brand | gaming pc – product<br>geforce – brand<br>geforce rtx 4090 – product<br>nvidia – brand | geforce rtx 4090 graphics card – product<br>nvidia – brand | geforce rtx 4090 – product<br>nvidia – brand | geforce rtx 4090 – product<br>nvidia – brand | geforce rtx 4090 – product<br>nvidia – brand |
| 25 | He's eating a McDonald's Big Mac with a side of Heinz ketchup. | big mac – product<br>heinz – brand<br>ketchup – product<br>mcdonald's – brand | big mac – product<br>heinz – brand<br>ketchup – product<br>mcdonald's – brand | big mac – product<br>heinz – brand<br>ketchup – product<br>mcdonald's – brand | big mac – product<br>heinz – brand<br>ketchup – product<br>mcdonald's – brand | big mac – product<br>heinz – brand<br>mcdonald's – brand | big mac – product<br>heinz – brand<br>ketchup – product<br>mcdonald's – brand |
| 26 | I'm writing notes in a Moleskine notebook with a Pilot G2 pen. | moleskine – brand<br>pilot g2 – product | moleskine – brand<br>moleskine notebook – product<br>pilot – brand<br>pilot g2 pen – product | moleskine – brand<br>notebook – product<br>pen – product<br>pilot – brand | g2 – product<br>moleskine – brand<br>notebook – product<br>pen – product<br>pilot – brand | moleskine – brand<br>pilot g2 – product | g2 pen – product<br>moleskine – brand<br>notebook – product<br>pilot – brand |
| 27 | She's using a Dyson Supersonic hair dryer. | dyson – brand<br>dyson supersonic – product<br>hair dryer – product | dyson – brand<br>supersonic hair dryer – product | dyson – brand<br>supersonic hair dryer – product | dyson – brand<br>supersonic hair dryer – product | dyson – brand<br>supersonic – product | dyson – brand<br>supersonic hair dryer – product |
| 28 | We're using Adobe Photoshop and Illustrator for our design project. | adobe – brand<br>adobe illustrator – product<br>adobe photoshop – product | adobe – brand<br>illustrator – product<br>photoshop – product | adobe – brand<br>illustrator – product<br>photoshop – product | adobe – brand<br>illustrator – product<br>photoshop – product | adobe – brand<br>illustrator – product<br>photoshop – product | adobe – brand<br>illustrator – product<br>photoshop – product |
| 29 | I'm wearing a Rolex Submariner watch. | rolex – brand<br>submariner – product | rolex – brand<br>submariner watch – product | rolex – brand<br>submariner watch – product | rolex – brand<br>submariner – product | rolex – brand<br>submariner – product | rolex – brand<br>submariner watch – product |
| 30 | He's playing Call of Duty on his Xbox Series X. | call of duty – product<br>xbox series x – product | call of duty – product<br>xbox – brand<br>xbox series x – product | call of duty – brand<br>xbox series x – product | call of duty – product<br>xbox series x – product | call of duty – product<br>xbox series x – product | call of duty – product<br>xbox – brand<br>xbox series x – product |
| 31 | I'm drinking a bottle of Evian water. | bottle of evian water – product<br>evian – brand | bottle – product<br>evian – brand<br>evian water – product | evian – brand<br>water – product | evian – brand<br>water – product | evian – brand<br>water – product | evian – brand<br>water – product |
| 32 | She's using a KitchenAid mixer to bake a cake. | kitchenaid – brand<br>mixer – product | kitchenaid – brand<br>mixer – product | kitchenaid – brand<br>mixer – product | cake – product<br>kitchenaid – brand<br>mixer – product | kitchenaid – brand<br>mixer – product | kitchenaid – brand<br>mixer – product |
| 33 | We're shopping for groceries at Walmart and Whole Foods. | grocery – product<br>walmart – brand<br>whole foods – brand | grocery – product<br>walmart – brand<br>whole foods – brand | grocery – product<br>walmart – brand<br>whole foods – brand | walmart – brand<br>whole foods – brand | walmart – brand<br>whole foods – brand | walmart – brand<br>whole foods – brand |
| 34 | I'm using a Weber grill for the barbecue this weekend. | grill – product<br>weber – brand | grill – product<br>weber – brand | grill – product<br>weber – brand | grill – product<br>weber – brand | grill – product<br>weber – brand | grill – product<br>weber – brand |
| 35 | He's taking an Uber to the airport to catch a British Airways flight. | british airways – brand<br>british airways flight – product<br>uber – brand<br>uber – product | british airways – brand<br>british airways flight – product<br>uber – brand | british airways – brand<br>flight – product<br>uber – brand | british airways – brand<br>uber – brand | british airways – brand<br>uber – brand | british airways – brand<br>uber – brand |
| 36 | I'm using a Herman Miller Aeron chair in my home office. | aeron chair – product<br>herman miller – brand | aeron chair – product<br>herman miller – brand | aeron chair – product<br>herman miller – brand | aeron chair – product<br>herman miller – brand | aeron – product<br>herman miller – brand | aeron chair – product<br>herman miller – brand |
| 37 | She's using a L'Oréal shampoo and a Dove soap. | dove – brand<br>l'oréal – brand<br>shampoo – product<br>soap – product | dove – brand<br>l'oreal – brand<br>shampoo – product<br>soap – product | dove – brand<br>l'oréal – brand<br>shampoo – product<br>soap – product | dove – brand<br>l'oréal – brand<br>shampoo – product<br>soap – product | dove – brand<br>l'oreal – brand<br>shampoo – product<br>soap – product | dove – brand<br>l'oréal – brand<br>shampoo – product<br>soap – product |
| 38 | We're using Slack and Zoom for team communication. | slack – brand<br>zoom – brand | slack – brand<br>slack – product<br>zoom – brand<br>zoom – product | slack – brand<br>zoom – brand | slack – brand<br>zoom – brand | slack – brand<br>zoom – brand | slack – brand<br>zoom – brand |
| 39 | I'm eating a bag of Lay's potato chips. | lay's – brand<br>potato chip – product | lay's – brand<br>potato chip – product | lays – brand<br>potato chip – product | lay's – brand<br>potato chip – product | lay's – brand<br>potato chips – product | lay's – brand<br>potato chip – product |
| 40 | He's driving a Toyota Prius, which is a popular hybrid car. | prius – product<br>toyota – brand | hybrid car – product<br>prius – product<br>toyota – brand | prius – product<br>toyota – brand | prius – product<br>toyota – brand | prius – product<br>toyota – brand | prius – product<br>toyota – brand |
| 41 | I'm using a GoPro HERO11 to record my vacation. | gopro – brand<br>hero11 – product | gopro – brand<br>hero11 – product | gopro – brand<br>hero11 – product | gopro – brand<br>hero11 – product | gopro – brand<br>hero11 – product | gopro – brand<br>hero11 – product |
| 42 | She's wearing a pair of Ray-Ban sunglasses. | ray-ban – brand<br>sunglasses – product | ray-ban – brand<br>sunglass – product | ray-ban – brand<br>sunglasses – product | ray-ban – brand<br>sunglasses – product | ray-ban – brand<br>sunglasses – product | ray-ban – brand<br>sunglasses – product |
| 43 | We're using a Roomba to clean the house. | roomba – brand | roomba – product | roomba – brand<br>roomba – product | roomba – brand | house – product<br>roomba – brand | roomba – brand<br>roomba – product |
| 44 | I'm using a Fitbit to track my steps and heart rate. | fitbit – brand | fitbit – brand<br>fitbit – product | fitbit – brand<br>fitbit – product | fitbit – brand | fitbit – brand<br>heart rate – product<br>step – product | fitbit – brand<br>fitbit – product |
| 45 | He's drinking a Red Bull energy drink. | red bull – brand<br>red bull energy drink – product | energy drink – product<br>red bull – brand | energy drink – product<br>red bull – brand | energy drink – product<br>red bull – brand | energy drink – product<br>red bull – brand | energy drink – product<br>red bull – brand |
| 46 | I'm using a Yeti tumbler to keep my coffee hot. | tumbler – product<br>yeti – brand | yeti – brand<br>yeti tumbler – product | tumbler – product<br>yeti – brand | tumbler – product<br>yeti – brand | tumbler – product<br>yeti – brand | tumbler – product<br>yeti – brand |
| 47 | She's using a MAC lipstick and a Fenty Beauty foundation. | fenty beauty – brand<br>foundation – product<br>lipstick – product<br>mac – brand | fenty beauty – brand<br>foundation – product<br>lipstick – product<br>mac – brand | fenty beauty – brand<br>foundation – product<br>lipstick – product<br>mac – brand | fenty beauty – brand<br>foundation – product<br>lipstick – product<br>mac – brand | fenty beauty – brand<br>foundation – product<br>lipstick – product<br>mac – brand | fenty beauty – brand<br>foundation – product<br>lipstick – product<br>mac – brand |
| 48 | We're using a Nest thermostat to control the temperature in our home. | nest – brand<br>nest thermostat – product | nest – brand<br>nest thermostat – product | nest – brand<br>thermostat – product | nest – brand<br>thermostat – product | nest – brand<br>thermostat – product | nest – brand<br>thermostat – product |
| 49 | I'm eating a bowl of Ben & Jerry's ice cream. | ben & jerry's – brand<br>ice cream – product | ben & jerry's – brand<br>ice cream – product | ben & jerry's – brand<br>ice cream – product | ben & jerry's – brand<br>ice cream – product | ben & jerry's – brand | ben & jerry's – brand<br>ice cream – product |
| 50 | He's using a Fender Stratocaster guitar to play a song by The Beatles. | fender – brand<br>stratocaster – product<br>the beatles – brand | fender – brand<br>guitar – product<br>stratocaster – product | fender – brand<br>stratocaster guitar – product | fender – brand<br>stratocaster – product<br>the beatles – brand | fender – brand<br>stratocaster – product<br>the beatles – brand | fender – brand<br>stratocaster guitar – product<br>the beatles – brand |

# Long answers

In [9]:
const aiQueries = [
    "What are the top language academies in Europe?",
    "Which car brands have the best safety ratings in 2024?",
    "What are the leading smartphone manufacturers by market share?",
    "Which pharmaceutical companies are developing COVID-19 vaccines?",
    "What are the most popular electric vehicle models in 2023?",
    "Which fashion brands are known for sustainable practices?",
    "What are the top social media platforms by user base?",
    "Which airlines offer the best in-flight services?",
    "What are the leading tech companies in artificial intelligence?",
    "Which food brands have the healthiest product lines?"
]

In [10]:
import { searchBatch } from "../src/search.ts";

const searchResults = await searchBatch({ prompts: aiQueries, model: "gpt-4.1", useSearch: true });
const answers = searchResults.map((result) => result.answer);

In [16]:
let ifExists: string = 'overwrite';
let execModels = models;

let longRuns = [];

for (const model of execModels) {
    const label = model.label ?? model.name;
    const exists = longRuns.findIndex((run) => run.model === label);
    if (exists != -1) {
        if (ifExists === 'ignore') {
            console.log(`Skipping ${label}, already run.`);
            continue;
        } else {
            console.log(`Overwriting previous run for ${label}.`);
        }
    }
    console.log(`Running ${label}...`);
    const startedAt = performance.now();
    const entities = await extractEntitiesBatch(
        answers,
        brandDefinitions,
        model.name,
        100,
        model.options
    );
    const durationMs = performance.now() - startedAt;
    const run = {
        model: label,
        entities,
        durationMs
    };
    if (exists != -1) {
        longRuns[exists] = run;
    } else {
        longRuns.push(run);
    }
}

Running GPT-5.1 (none)...
Running GPT-5 Mini (low)...
Running GPT-5 Nano (low)...
Running gpt-4.1...
Running gpt-4.1-mini...
Running gpt-4.1-nano...


6

In [17]:
const comparisons = answers.map((text, index) => ({
    index,
    text,
    entitiesByModel: longRuns.map((run) => ({
        model: run.model,
        entities: run.entities[index] ?? []
    }))
}));

const processingTimes = longRuns.map((run) => ({
    model: run.model,
    durationMs: run.durationMs
}));

processingTimes;

[
  { model: "GPT-5.1 (none)", durationMs: 6960.349625000032 },
  { model: "GPT-5 Mini (low)", durationMs: 22962.37291699997 },
  { model: "GPT-5 Nano (low)", durationMs: 17077.531624999945 },
  { model: "gpt-4.1", durationMs: 9835.831458 },
  { model: "gpt-4.1-mini", durationMs: 8001.218583000009 },
  { model: "gpt-4.1-nano", durationMs: 9298.24329100002 }
]

In [ ]:
function escapeForMarkdown(text: string): string {
    return text.replace(/\|/g, "\\|").replace(/\n/g, "<br>");
}

const headerCells = [
    "#",
    "Sentence",
    ...longRuns.flatMap((run) => [`${run.model} entities`])
];
const header = `| ${headerCells.join(" | ")} |`;
const separator = `| ${headerCells.map(() => "---").join(" | ")} |`;

const rows = comparisons.map((row) => {
    const cells = [
        String(row.index + 1),
        escapeForMarkdown(row.text ?? "")
    ];

    for (const entry of row.entitiesByModel) {
        // const names = escapeForMarkdown(entry.entities.sort().map((entity) => entity.name + " – " + entity.type).join("\n"));
        const names = escapeForMarkdown(
            entry.entities
                .sort((a, b) => a.name.localeCompare(b.name))
                .map((entity) => entity.name + " – " + entity.type)
                .join("\n")
        );
        cells.push(names);
    }

    return `| ${cells.join(" | ")} |`;
});

const markdownTable = [header, separator, ...rows].join("\n");

md`${markdownTable}`;

| # | Sentence | GPT-5.1 (none) entities | GPT-5 Mini (low) entities | GPT-5 Nano (low) entities | gpt-4.1 entities | gpt-4.1-mini entities | gpt-4.1-nano entities |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | Here are some of the top language academies and schools across Europe, based on recognized academic excellence, breadth of offerings, and student experiences:<br><br>• **INALCO (Institut national des langues et civilisations orientales), Paris, France**  <br>  This prestigious public institution, founded in 1669, is considered the world’s largest provider of language training with 104 languages taught as of 2024. It offers programs ranging from Bachelor’s to PhD, as well as continuing education and short courses in diverse global languages ([en.wikipedia.org](https://en.wikipedia.org/wiki/Institut_national_des_langues_et_civilisations_orientales?utm_source=openai)).<br><br>• **Lycée International de Saint-Germain-en-Laye, France**  <br>  One of France's most prestigious bilingual schools, offering the French national curriculum alongside international sections in 14 languages. It prepares students for both the Baccalauréat and the IB Diploma ([rachaelsknowledge.com](https://rachaelsknowledge.com/the-best-bilingual-schools-in-europe/?utm_source=openai)).<br><br>• **The British School of Brussels, Belgium**  <br>  A well-regarded bilingual English–French school with IB curriculum offerings. Known for strong academic standards and modern facilities ([rachaelsknowledge.com](https://rachaelsknowledge.com/the-best-bilingual-schools-in-europe/?utm_source=openai)).<br><br>• **Deutsche Schule Madrid, Spain**  <br>  A German–Spanish bilingual institution offering the German Abitur. German, English, and French are also available, with consistently high university acceptance rates across Europe ([rachaelsknowledge.com](https://rachaelsknowledge.com/the-best-bilingual-schools-in-europe/?utm_source=openai)).<br><br>• **Collège du Léman, Switzerland**  <br>  Offers a bilingual English–French curriculum with options for IB and Swiss Maturité. Known for its multicultural student body (120+ nationalities), excellent facilities, and boarding provisions ([rachaelsknowledge.com](https://rachaelsknowledge.com/the-best-bilingual-schools-in-europe/?utm_source=openai)).<br><br>• **International School of Geneva, Switzerland**  <br>  The oldest international school globally, offering bilingual IB programs in English and French, with outstanding academic results and university placements ([rachaelsknowledge.com](https://rachaelsknowledge.com/the-best-bilingual-schools-in-europe/?utm_source=openai)).<br><br>• **St. Stephen’s School, Rome, Italy**  <br>  English–Italian bilingual school offering the IB curriculum, with a strong focus on humanities and international relations. Smaller class sizes support personalized learning ([rachaelsknowledge.com](https://rachaelsknowledge.com/the-best-bilingual-schools-in-europe/?utm_source=openai)).<br><br>• **International School of Paris, France**  <br>  A fully bilingual institution offering both IB and French Baccalaureate options. Recognized for excellent support for non-French speakers and fostering cross-cultural understanding and critical thinking ([rachaelsknowledge.com](https://rachaelsknowledge.com/the-best-bilingual-schools-in-europe/?utm_source=openai)).<br><br>• **Well-known academies for as-needed language learning across Europe**  <br>  Several platforms highlight highly reputable language schools tailored to individual short- or medium-term needs:<br>  - In **Spain**, **IH Madrid**, **Enforex Barcelona**, and **don Quijote Madrid** consistently rank among the top for Spanish language study ([smapse.com](https://smapse.com/catalog/english/language-school/?utm_source=openai)).<br>  - Language education platforms such as **Go Find Programs** offer strong course options in popular cities: German in Berlin, Spanish in Valencia or Barcelona, French in Montpellier, and Italian in Bologna ([gofindprograms.com](https://www.gofindprograms.com/language-schools/europe?utm_source=openai)).<br>  - **English Path** currently operates acclaimed English, French, and German language programs in **Malta** and **Dublin**, with future expansion planned to Berlin and Paris ([englishpath.com](https://www.englishpath.com/destinations/europe/?utm_source=openai)).<br><br>---  <br><br>**Summary Table (select institutions across categories):**<br><br>- **Higher-education institution (multi-language)**: INALCO – Paris, France  <br>- **Prestigious bilingual secondary schools**:  <br>  - Lycée International de Saint-Germain-en-Laye – France  <br>  - British School of Brussels – Belgium  <br>  - Deutsche Schule Madrid – Spain  <br>  - Collège du Léman – Switzerland  <br>  - International School of Geneva – Switzerland  <br>  - St. Stephen’s School – Italy  <br>  - International School of Paris – France  <br>- **Language-specific academies for intensive learning**:  <br>  - IH Madrid, Enforex Barcelona, don Quijote Madrid – Spain  <br>  - Go Find Programs (multiple European cities)  <br>  - English Path (Malta, Dublin; soon Berlin/Paris)<br><br>If you'd like a more tailored list—whether by country, language specialty (e.g., English, Spanish, German, Arabic), or program type (intensive, immersion, academic credentials)—let me know and I’ll be glad to refine the recommendations to suit your needs. | collège du léman – brand<br>deutsche schule madrid – brand<br>don quijote madrid – brand<br>enforex barcelona – brand<br>english path – brand<br>go find programs – brand<br>ih madrid – brand<br>inalco – brand<br>institut national des langues et civilisations orientales – brand<br>international school of geneva – brand<br>international school of paris – brand<br>lycée international de saint-germain-en-laye – brand<br>st. stephen’s school – brand<br>the british school of brussels – brand | baccalauréat – product<br>british school of brussels – brand<br>collège du léman – brand<br>continuing education – product<br>deutsche schule madrid – brand<br>don quijote madrid – brand<br>enforex barcelona – brand<br>english path – brand<br>german abitur – product<br>go find programs – brand<br>ib diploma – product<br>ib program – product<br>ih madrid – brand<br>immersion program – product<br>inalco – brand<br>intensive program – product<br>international school of geneva – brand<br>international school of paris – brand<br>language training – product<br>lycée international de saint-germain-en-laye – brand<br>short course – product<br>st. stephen’s school – brand<br>swiss maturité – product | british school of brussels – brand<br>college du léman – brand<br>deutsche schule madrid – brand<br>don quijote madrid – brand<br>enforex barcelona – brand<br>english path – brand<br>go find programs – brand<br>ih madrid – brand<br>inalco – brand<br>international school of geneva – brand<br>international school of paris – brand<br>lycee international de saint-germain-en-laye – brand<br>st stephens school – brand | baccalauréat – product<br>bachelor’s – product<br>collège du léman – brand<br>continuing education – product<br>deutsche schule madrid – brand<br>don quijote madrid – brand<br>enforex barcelona – brand<br>english path – brand<br>french baccalaureate – product<br>french national curriculum – product<br>german abitur – product<br>go find programs – brand<br>ib – product<br>ib diploma – product<br>ih madrid – brand<br>inalco – brand<br>international school of geneva – brand<br>international school of paris – brand<br>language training – product<br>lycée international de saint-germain-en-laye – brand<br>phd – product<br>short course – product<br>st. stephen’s school – brand<br>swiss maturité – product<br>the british school of brussels – brand | baccalauréat – product<br>british school of brussels – brand<br>collège du léman – brand<br>deutsche schule madrid – brand<br>don quijote madrid – brand<br>enforex barcelona – brand<br>english path – brand<br>french baccalaureate – product<br>german abitur – product<br>go find programs – brand<br>ib – product<br>ib curriculum – product<br>ib diploma – product<br>ih madrid – brand<br>inalco – brand<br>international school of geneva – brand<br>international school of paris – brand<br>lycée international de saint-germain-en-laye – brand<br>st. stephen’s school – brand<br>swiss maturité – product | collège du léman – brand<br>deutsche schule madrid – brand<br>don quijote madrid – brand<br>enforex barcelona – brand<br>english path – brand<br>gofindprograms – brand<br>ih madrid – brand<br>inalco – brand<br>institut national des langues et civilisations orientales – product<br>international school of geneva – brand<br>international school of paris – brand<br>lycée international de saint-germain-en-laye – brand<br>st. stephen's school – brand<br>the british school of brussels – brand |
| 2 | Here’s a thorough look at which car brands had the **best safety ratings in 2024**, focusing on U.S.-based evaluation systems: the **Insurance Institute for Highway Safety (IIHS)** and, to a lesser extent, the **National Highway Traffic Safety Administration (NHTSA)**.<br><br>---<br><br>##  IIHS Top Safety Picks—2024 Overview<br><br>### Hyundai Motor Group Leads the Pack<br>- Hyundai Motor Group—which includes **Hyundai**, **Kia**, and **Genesis**—earned the most Top Safety Pick awards in 2024 with a total of **16** (6 Top Safety Pick+ and 10 Top Safety Pick) ([forbes.com](https://www.forbes.com/sites/tanyamohn/2024/02/28/the-safest-cars-of-2024--hyundaitoyota-and-mazda-among-standouts/?utm_source=openai)).<br>- The breakdown:<br>  - **Hyundai**: several models such as the Ioniq 6 and Kona.<br>  - **Genesis**: Electrified G80, GV60, and GV80 among the Top Safety Pick+ recipients ([cars.com](https://www.cars.com/articles/these-71-cars-earned-iihs-top-safety-pick-awards-amid-tougher-criteria-478644/?utm_source=openai)).<br>  - **Kia**: Telluride among the higher-tier winners ([cars.com](https://www.cars.com/articles/these-71-cars-earned-iihs-top-safety-pick-awards-amid-tougher-criteria-478644/?utm_source=openai)).<br><br>### Toyota Motor Corporation Close Behind<br>- Toyota (including **Toyota** and **Lexus**) secured **1 Top Safety Pick+** and **12 Top Safety Pick** awards, for a combined total of **13** ([forbes.com](https://www.forbes.com/sites/tanyamohn/2024/02/28/the-safest-cars-of-2024--hyundaitoyota-and-mazda-among-standouts/?utm_source=openai)).<br><br>### Mazda Shows Strong Performance with Fewer Models<br>- Mazda earned **5 Top Safety Pick+** and **1 Top Safety Pick** awards — significant considering its smaller vehicle lineup ([autosphere.ca](https://autosphere.ca/fleet/2024/04/05/the-iihs-presents-its-2024-ranking-of-safest-vehicles/?utm_source=openai)).<br><br>---<br><br>##  Other Notable Brands and Model Highlights<br><br>- **Acura**: Integra and MDX achieved Top Safety Pick+ awards ([cars.com](https://www.cars.com/articles/these-71-cars-earned-iihs-top-safety-pick-awards-amid-tougher-criteria-478644/?utm_source=openai)).<br>- **BMW**, **Mercedes-Benz**, **Honda**, **Subaru**, **Tesla**, **Nissan**, and others also had standout models earning either Top Safety Pick+ (e.g., BMW X3, Mercedes-Benz GLE, Honda Accord) or Top Safety Pick distinctions ([cars.com](https://www.cars.com/articles/these-71-cars-earned-iihs-top-safety-pick-awards-amid-tougher-criteria-478644/?utm_source=openai)).<br>- At the end of 2024, additional models from **Ford** (Mustang Mach-E), **Infiniti QX60**, **Mazda CX-70**, **Rivian R1S**, **Subaru Forester**, **Toyota Tundra**, and **Volvo XC90 Plug‑In Hybrid** were added to the Top Safety Pick+ list ([iihs.org](https://www.iihs.org/news/detail/nineteen-more-vehicles-close-out-2024-with-iihs-awards?utm_source=openai)).<br><br>---<br><br>##  Broader Brand Rankings—IIHS Awards Summary<br><br>A further snapshot of overall brand standings across both Top Safety Pick+ and Top Safety Pick categories shows:<br><br>- **Hyundai Motor Group** (Hyundai, Kia, Genesis): 7 Top Safety Pick+ awards, 7 Top Safety Pick — total **14** (though earlier data suggests 16, this may exclude late-year additions) ([autosphere.ca](https://autosphere.ca/fleet/2024/04/05/the-iihs-presents-its-2024-ranking-of-safest-vehicles/?utm_source=openai)).<br>- **Toyota**: 1 Top Safety Pick+, 7 Top Safety Pick — total **8** ([autosphere.ca](https://autosphere.ca/fleet/2024/04/05/the-iihs-presents-its-2024-ranking-of-safest-vehicles/?utm_source=openai)).<br>- **Mazda**: 5 Top Safety Pick+, 1 Top Safety Pick — total **6** ([autosphere.ca](https://autosphere.ca/fleet/2024/04/05/the-iihs-presents-its-2024-ranking-of-safest-vehicles/?utm_source=openai)).<br>- **Honda**: 2 Top Safety Pick+, 3 Top Safety Pick — total **5** ([autosphere.ca](https://autosphere.ca/fleet/2024/04/05/the-iihs-presents-its-2024-ranking-of-safest-vehicles/?utm_source=openai)).<br>- **Subaru**: 1 Top Safety Pick+, 3 Top Safety Pick — total **4** ([autosphere.ca](https://autosphere.ca/fleet/2024/04/05/the-iihs-presents-its-2024-ranking-of-safest-vehicles/?utm_source=openai)).<br>- Other contributors include **Acura**, **BMW**, **Mercedes-Benz**, **Tesla**, etc., with between 2–3 awards each ([autosphere.ca](https://autosphere.ca/fleet/2024/04/05/the-iihs-presents-its-2024-ranking-of-safest-vehicles/?utm_source=openai)).<br><br>---<br><br>##  NHTSA 5-Star Ratings<br><br>While the NHTSA's 5-Star Safety Ratings are influential, they serve more as a broad, initial benchmark rather than listing top brands. Their 2024 model year crash tests cover many makes, but they **do not compile an overall ranking by brand** ([nhtsa.gov](https://www.nhtsa.gov/press-releases/nhtsa-2024-crash-test-vehicles-5-star-safety-ratings?utm_source=openai)).<br><br>---<br><br>## Summary: Which Brands Offered the Best Safety in 2024?<br><br>- **Hyundai Motor Group** (Hyundai, Genesis, Kia) stood out with the highest volume of IIHS safety awards.<br>- **Toyota** (including Lexus) followed closely with strong performance across multiple models.<br>- **Mazda** earned notable recognition, particularly for excellence across a smaller range of vehicles.<br>- **Other respected brands** include Honda, Subaru, Acura, BMW, Mercedes-Benz, and Tesla—each had multiple models meeting stringent safety criteria.<br>- Overall, vehicle safety in 2024 was spearheaded by these automakers, as validated by the rigorous IIHS Top Safety Pick and Top Safety Pick+ designations. | acura – brand<br>bmw – brand<br>cx-70 – product<br>electrified g80 – product<br>ford – brand<br>forester – product<br>genesis – brand<br>gv60 – product<br>gv80 – product<br>honda – brand<br>hyundai – brand<br>hyundai motor group – brand<br>infiniti – brand<br>integra – product<br>ioniq 6 – product<br>kia – brand<br>kona – product<br>lexus – brand<br>mazda – brand<br>mdx – product<br>mercedes-benz – brand<br>mustang mach-e – product<br>nissan – brand<br>qx60 – product<br>r1s – product<br>rivian – brand<br>subaru – brand<br>telluride – product<br>tesla – brand<br>toyota – brand<br>toyota motor corporation – brand<br>tundra – product<br>volvo – brand<br>xc90 plug‑in hybrid – product | acura – brand<br>acura integra – product<br>acura mdx – product<br>bmw – brand<br>bmw x3 – product<br>ford – brand<br>ford mustang mach-e – product<br>genesis – brand<br>genesis electrified g80 – product<br>genesis gv60 – product<br>genesis gv80 – product<br>honda – brand<br>honda accord – product<br>hyundai – brand<br>hyundai ioniq 6 – product<br>hyundai kona – product<br>hyundai motor group – brand<br>infiniti – brand<br>infiniti qx60 – product<br>kia – brand<br>kia telluride – product<br>lexus – brand<br>mazda – brand<br>mazda cx-70 – product<br>mercedes-benz – brand<br>mercedes-benz gle – product<br>nissan – brand<br>rivian – brand<br>rivian r1s – product<br>subaru – brand<br>subaru forester – product<br>tesla – brand<br>toyota – brand<br>toyota motor corporation – brand<br>toyota tundra – product<br>volvo – brand<br>volvo xc90 plug-in hybrid – product | accord – product<br>acura – brand<br>bmw – brand<br>electrified g80 – product<br>ford – brand<br>genesis – brand<br>gle – product<br>gv60 – product<br>gv80 – product<br>honda – brand<br>hyundai – brand<br>hyundai motor group – brand<br>infiniti – brand<br>integra – product<br>ioniq 6 – product<br>kia – brand<br>kona – product<br>lexus – brand<br>mazda – brand<br>mazda cx-70 – product<br>mdx – product<br>mercedes-benz – brand<br>mustang mach-e – product<br>nissan – brand<br>qx60 – product<br>rivian – brand<br>rivian r1s – product<br>subaru – brand<br>subaru forester – product<br>telluride – product<br>tesla – brand<br>toyota – brand<br>toyota tundra – product<br>volvo – brand<br>volvo xc90 plug-in hybrid – product<br>x3 – product | accord – product<br>acura – brand<br>bmw – brand<br>cx-70 – product<br>electrified g80 – product<br>ford – brand<br>forester – product<br>genesis – brand<br>gle – product<br>gv60 – product<br>gv80 – product<br>honda – brand<br>hyundai – brand<br>hyundai motor group – brand<br>infiniti – brand<br>integra – product<br>ioniq 6 – product<br>kia – brand<br>kona – product<br>lexus – brand<br>mazda – brand<br>mdx – product<br>mercedes-benz – brand<br>mustang mach-e – product<br>nissan – brand<br>qx60 – product<br>r1s – product<br>rivian – brand<br>subaru – brand<br>telluride – product<br>tesla – brand<br>toyota – brand<br>tundra – product<br>volvo – brand<br>x3 – product<br>xc90 plug‑in hybrid – product | acura – brand<br>bmw – brand<br>bmw x3 – product<br>electrified g80 – product<br>ford – brand<br>genesis – brand<br>gv60 – product<br>gv80 – product<br>honda – brand<br>honda accord – product<br>hyundai – brand<br>hyundai motor group – brand<br>infiniti qx60 – product<br>integra – product<br>ioniq 6 – product<br>kia – brand<br>kona – product<br>lexus – brand<br>mazda – brand<br>mazda cx-70 – product<br>mdx – product<br>mercedes-benz – brand<br>mercedes-benz gle – product<br>mustang mach-e – product<br>nissan – brand<br>rivian r1s – product<br>subaru – brand<br>subaru forester – product<br>telluride – product<br>tesla – brand<br>toyota – brand<br>toyota motor corporation – brand<br>toyota tundra – product<br>volvo xc90 plug-in hybrid – product | accord – product<br>acura – brand<br>bmw – brand<br>cx-70 – product<br>ford – brand<br>forester – product<br>genesis – brand<br>gle – product<br>gv60 – product<br>gv80 – product<br>honda – brand<br>hyundai – brand<br>infiniti – brand<br>integra – product<br>ioniq 6 – product<br>kia – brand<br>kona – product<br>lexus – brand<br>mazda – brand<br>mdx – product<br>mercedes-benz – brand<br>mustang mach-e – product<br>nissan – brand<br>qx60 – product<br>r1s – product<br>rivian – brand<br>subaru – brand<br>telluride – product<br>tesla – brand<br>toyota – brand<br>tundra – product<br>volvo – brand<br>x3 – product<br>xc90 plug-in hybrid – product<br>xc90 plug-in hybrid – product |
| 3 | Here is an updated overview (as of Q3 2025) of the **leading smartphone manufacturers by global market share**, highlighting the top vendors and their performance:<br><br>---<br><br>##  Global Smartphone Market Leaders (Q3 2025)<br><br>Based on the latest data from market research firms like Omdia and others:<br><br>- **Samsung**: Held the top position globally in Q3 2025 with approximately **60.6 million units shipped**, translating to a **19% market share**, and achieved around **6% year-over-year growth** ([windpress.info](https://windpress.info/it/press-release/933257/omdia-global-smartphone-market-grows-3-in-q3-2025-led-by-emerging-economies?utm_source=openai)).<br>  <br>- **Apple**: Secured the second spot with **56.5 million shipments** (~**18% market share**) and saw about **4% YoY growth** ([windpress.info](https://windpress.info/it/press-release/933257/omdia-global-smartphone-market-grows-3-in-q3-2025-led-by-emerging-economies?utm_source=openai)).<br><br>- **Xiaomi**: Ranked third, shipping around **43.4 million units** (~**14% share**), with modest **1% annual growth** ([windpress.info](https://windpress.info/it/press-release/933257/omdia-global-smartphone-market-grows-3-in-q3-2025-led-by-emerging-economies?utm_source=openai)).<br><br>- **Transsion**: Positioned fourth with **28.6 million units** (~**9% share**), demonstrating strong **12% YoY growth**, largely driven by African markets (Infinix, TECNO, itel) ([windpress.info](https://windpress.info/it/press-release/933257/omdia-global-smartphone-market-grows-3-in-q3-2025-led-by-emerging-economies?utm_source=openai)).<br><br>- **vivo**: Came in fifth, shipping approximately **28.5 million units** (~**9% share**) and delivering **5% growth**, with performance especially strong in India and Southeast Asia ([windpress.info](https://windpress.info/it/press-release/933257/omdia-global-smartphone-market-grows-3-in-q3-2025-led-by-emerging-economies?utm_source=openai)).<br><br>Another source, Accio, presents similar figures with slight variation—reporting Samsung at **19.0%**, Apple at **18.2%**, Xiaomi at **13.5%**, Transsion at **9.0%**, and vivo at **8.9%** share ([accio.com](https://www.accio.com/t-v2/business/smartphone-brands-market-share-trend?utm_source=openai)).<br><br>---<br><br>##  Contextual Insights & Regional Highlights<br><br>- The global smartphone market saw a modest rebound in Q3 2025, with shipments totaling around **320 million units**, marking a **3% year-over-year increase** ([omdia.tech.informa.com](https://omdia.tech.informa.com/pr/2025/oct/omdia-global-smartphone-market-grows-3percent-in-q3-2025-led-by-emerging-economies?utm_source=openai)).<br><br>- **Emerging markets** played a significant role in fueling growth, especially regions like Africa and the Middle East, which experienced double-digit increases ([techinsights.com](https://www.techinsights.com/blog/narrative-global-smartphone-vendor-and-os-share-region-q3-2025?utm_source=openai)).<br><br>- Analysts attributed Samsung’s strong performance to its wide-ranging portfolio—from high-end foldable devices (Galaxy Z Fold7, Z Flip7) to mid- and low-range Galaxy A series models driving demand in Asia-Pacific and Middle East ([androidheadlines.com](https://www.androidheadlines.com/2025/10/global-smartphone-market-q3-2025-samsung-share.html?utm_source=openai)).<br><br>- Apple's year-over-year growth was largely boosted by early demand for its iPhone 17 lineup, with strategic pricing and storage improvements contributing to its Q3 performance ([hardwarezone.com.sg](https://www.hardwarezone.com.sg/mobile/smartphones/omdia-smartphone-market-share-apple-strongest-q3-2025/?utm_source=openai)).<br><br>- Transsion’s continued ascendancy—especially in Africa—is notable as it leverages strong demand in entry-level smartphone segments ([true-tech.net](https://true-tech.net/smartphone-sales-q3-2025-report/?utm_source=openai)).<br><br>- Vivo maintained solid performance, especially in India and Southeast Asia, where its affordable and regionally optimized models remain popular ([true-tech.net](https://true-tech.net/smartphone-sales-q3-2025-report/?utm_source=openai)).<br><br>---<br><br>## Summary Table (Q3 2025 Leading Vendors)<br><br>\| Rank \| Manufacturer \| Approx. Shipments (Millions) \| Market Share \| YoY Growth \|<br>\|------\|--------------\|-------------------------------\|--------------\|------------\|<br>\| 1 \| Samsung \| 60.6 \| ~19% \| +6% \|<br>\| 2 \| Apple \| 56.5 \| ~18% \| +4% \|<br>\| 3 \| Xiaomi \| 43.4 \| ~14% \| +1% \|<br>\| 4 \| Transsion \| 28.6 \| ~9% \| +12% \|<br>\| 5 \| vivo \| 28.5 \| ~9% \| +5% \|<br><br>---<br><br>##  Final Thoughts<br><br>As of Q3 2025, the global smartphone market is led by Samsung and Apple, closely followed by Xiaomi, Transsion, and vivo. Samsung continues to benefit from its broad product portfolio, while Apple sees gains through premium offerings. Transsion and vivo, meanwhile, capitalize on their strong presence in emerging markets and price-sensitive segments.<br><br>If you’d like insights into regional breakdowns, operating system dominance, or longer-term trends, feel free to ask! | apple – brand<br>galaxy a series – product<br>galaxy z fold7 – product<br>infinix – brand<br>iphone 17 – product<br>itel – brand<br>samsung – brand<br>tecno – brand<br>transsion – brand<br>vivo – brand<br>xiaomi – brand<br>z flip7 – product | apple – brand<br>galaxy a series – product<br>galaxy z fold7 – product<br>infinix – brand<br>iphone 17 – product<br>itel – brand<br>samsung – brand<br>tecno – brand<br>transsion – brand<br>vivo – brand<br>xiaomi – brand<br>z flip7 – product | apple – brand<br>galaxy a series – product<br>galaxy z flip7 – product<br>galaxy z fold7 – product<br>infinix – brand<br>iphone 17 lineup – product<br>itel – brand<br>samsung – brand<br>tecno – brand<br>transsion – brand<br>vivo – brand<br>xiaomi – brand | apple – brand<br>galaxy a series – product<br>galaxy z fold7 – product<br>infinix – brand<br>iphone 17 – product<br>itel – brand<br>samsung – brand<br>tecno – brand<br>transsion – brand<br>vivo – brand<br>xiaomi – brand<br>z flip7 – product | apple – brand<br>galaxy a series – product<br>galaxy z fold7 – product<br>iphone 17 – product<br>samsung – brand<br>transsion – brand<br>vivo – brand<br>xiaomi – brand<br>z flip7 – product | apple – brand<br>galaxy a series – product<br>galaxy z flip7 – product<br>galaxy z fold7 – product<br>iphone 17 – product<br>samsung – brand<br>transsion – brand<br>vivo – brand<br>xiaomi – brand |
| 4 | Here is an updated overview (as of November 19, 2025) of pharmaceutical companies that **have developed** or are **actively developing** COVID‑19 vaccines. This includes those with authorized vaccines (globally or in the U.S.), as well as others currently in various stages of development.<br><br>**1. Pfizer/BioNTech**  <br>- Developed the mRNA-based COVID‑19 vaccine **Comirnaty**, jointly with BioNTech. It was the first mRNA COVID‑19 vaccine authorized globally. ([en.wikipedia.org](https://en.wikipedia.org/wiki/BioNTech?utm_source=openai))  <br>- Comirnaty remains FDA-approved for adults aged 65+, and for individuals aged 5–64 at high risk. ([reuters.com](https://www.reuters.com/business/healthcare-pharmaceuticals/which-covid-19-vaccines-are-fda-approved-us-2025-08-27/?utm_source=openai))  <br><br>**2. Moderna**  <br>- Developed the mRNA vaccine **Spikevax**, and more recently, a next-generation low-dose version called **mNEXSPIKE**, approved by the FDA (as of May 2025). ([apnews.com](https://apnews.com/article/84293ee6799dfbbe4e66dce06022afae?utm_source=openai))  <br>- Spikevax is approved for a broad range (from 6 months onward), while mNEXSPIKE is approved for older adults and high-risk individuals aged 12–64. ([apnews.com](https://apnews.com/article/84293ee6799dfbbe4e66dce06022afae?utm_source=openai))  <br><br>**3. Novavax (distributed by Sanofi)**  <br>- Developed **Nuvaxovid**, a protein-based COVID‑19 vaccine. It received FDA approval in May 2025 for adults 65+ and for individuals 12–64 with underlying conditions. ([investopedia.com](https://www.investopedia.com/novavax-stock-soars-on-fda-approval-for-new-covid-19-vaccine-11737357?utm_source=openai))  <br>- Novavax has commercialized the vaccine through a partnership with Sanofi, receiving milestone payments and licensing rights. ([reuters.com](https://www.reuters.com/business/healthcare-pharmaceuticals/novavax-beats-quarterly-revenue-estimates-milestone-payment-2025-08-06/?utm_source=openai))  <br><br>**4. Arcturus Therapeutics (in collaboration with Vinbiocare)**  <br>- Developed a **self‑amplifying mRNA** COVID‑19 vaccine named **Zapomeran** (also known as Kostaive).  <br>- It was approved for use in Japan (November 2023) and authorized in the European Union (February 2025). ([en.wikipedia.org](https://en.wikipedia.org/wiki/Zapomeran?utm_source=openai))  <br><br>**5. ImmunityBio (hAd5 vaccine)**  <br>- A U.S.-based pharmaceutical company developing a **viral-vector COVID‑19 vaccine**, codenamed **hAd5**. It is currently in Phase 1 clinical trials, with manufacturing support from South Africa’s Biovac for distribution across Africa. ([en.wikipedia.org](https://en.wikipedia.org/wiki/ImmunityBio_COVID-19_vaccine?utm_source=openai))  <br><br>**6. Walvax Biotech / Shanghai Zerun Biotechnology**  <br>- Developing **202‑CoV**, a protein-based COVID‑19 vaccine candidate currently in Phase 1 trials. Funding for the project ceased as of late 2022, but development continues. ([en.wikipedia.org](https://en.wikipedia.org/wiki/202-CoV?utm_source=openai))  <br><br>**Other noteworthy initiatives and technology transfer efforts:**  <br>- **Serum Institute of India (SII)** produced and distributed doses of AstraZeneca/Oxford and Novavax vaccines through COVAX agreements. ([who.int](https://www.who.int/news/item/18-12-2020-covax-announces-additional-deals-to-access-promising-covid-19-vaccine-candidates-plans-global-rollout-starting-q1-2021?utm_source=openai))  <br>- The **WHO’s mRNA Technology Transfer Programme** includes regional hubs like Afrigen (South Africa), Biovac, and others in low- and middle-income countries, enabling manufacturing and development of mRNA COVID‑19 vaccines (e.g., AfriVac 2121). ([who.int](https://www.who.int/initiatives/the-mrna-vaccine-technology-transfer-hub?utm_source=openai))  <br>- The **COVID-19 Technology Access Pool (C‑TAP)** facilitated voluntary licensing to manufacturers such as Medigen (Taiwan) and CSIC (Spain) to share COVID‑19 vaccine technology. ([who.int](https://www.who.int/news/item/29-08-2023-who-initiative-signs-new-licensing-agreements-on-covid-19-technologies?utm_source=openai))  <br>- **CanSino Biologics** (China) developed the viral vector vaccine **CONVIDECIA**, which received WHO emergency use listing in May 2022. ([who.int](https://www.who.int/news/item/19-05-2022-who-validates-11th-vaccine-for-covid-19?utm_source=openai))  <br><br>Summary list:<br>- Pfizer/BioNTech (mRNA – Comirnaty)  <br>- Moderna (mRNA – Spikevax; mNEXSPIKE)  <br>- Novavax (protein-based – Nuvaxovid)  <br>- Arcturus Therapeutics / Vinbiocare (self‑amplifying mRNA – Zapomeran/Kostaive)  <br>- ImmunityBio (viral-vector – hAd5)  <br>- Walvax Biotech / Shanghai Zerun (protein – 202‑CoV)  <br>- CanSino Biologics (viral-vector – CONVIDECIA)  <br><br>Additionally, multiple technology transfer and licensing initiatives are aiding vaccine availability and development globally.<br><br>If you’d like details on specific vaccine technologies, approvals in particular countries, or global collaborations, feel free to ask! | 202‑cov – product<br>afrivac 2121 – product<br>arcturus therapeutics – brand<br>astrazeneca – brand<br>biontech – brand<br>c‑tap – product<br>cansino biologics – brand<br>comirnaty – product<br>convidencia – product<br>covid-19 technology access pool – product<br>csic – brand<br>had5 – product<br>immunitybio – brand<br>kostaive – product<br>medigen – brand<br>mnexspike – product<br>moderna – brand<br>novavax – brand<br>nuvaxovid – product<br>oxford – brand<br>pfizer – brand<br>sanofi – brand<br>serum institute of india – brand<br>shanghai zerun biotechnology – brand<br>sii – brand<br>spikevax – product<br>vinbiocare – brand<br>walvax biotech – brand<br>who’s mrna technology transfer programme – product<br>zapomeran – product | 202-cov – product<br>afrigen – brand<br>afrivac 2121 – product<br>arcturus therapeutics – brand<br>biontech – brand<br>biovac – brand<br>cansino biologics – brand<br>comirnaty – product<br>convidecia – product<br>csic – brand<br>had5 – product<br>immunitybio – brand<br>kostaive – product<br>medigen – brand<br>mnexspike – product<br>moderna – brand<br>novavax – brand<br>nuvaxovid – product<br>pfizer – brand<br>sanofi – brand<br>serum institute of india – brand<br>shanghai zerun biotechnology – brand<br>spikevax – product<br>vinbiocare – brand<br>walvax biotech – brand<br>who – brand<br>zapomeran – product | 202-coV – product<br>arcturus therapeutics – brand<br>cansino biologics – brand<br>comirnaty – product<br>convidecia – product<br>had5 – product<br>immunitybio – brand<br>kostaive – product<br>mnexspike – product<br>novavax – brand<br>nuvaxovid – product<br>pfizer/biontech – brand<br>shanghai zerun biotechnology – brand<br>spikevax – product<br>vinbiocare – brand<br>walvax biotech – brand<br>zapomeran – product | 202-cov – product<br>afrigen – brand<br>afrivac 2121 – product<br>arcturus therapeutics – brand<br>astrazeneca – brand<br>biontech – brand<br>biovac – brand<br>c-tap – product<br>cansino biologics – brand<br>comirnaty – product<br>convidencia – product<br>covax – product<br>covid-19 technology access pool – product<br>csic – brand<br>had5 – product<br>immunitybio – brand<br>kostaive – product<br>medigen – brand<br>mnexspike – product<br>moderna – brand<br>mrna technology transfer programme – product<br>novavax – brand<br>nuvaxovid – product<br>oxford – brand<br>pfizer – brand<br>sanofi – brand<br>serum institute of india – brand<br>shanghai zerun biotechnology – brand<br>spikevax – product<br>vinbiocare – brand<br>walvax biotech – brand<br>who – brand<br>zapomeran – product | 202cov – product<br>afribac 2121 – product<br>africell – brand<br>arcturus therapeutics – brand<br>astrazeneca – brand<br>biontech – brand<br>cansino biologics – brand<br>comirnaty – product<br>convidencia – product<br>covid-19 technology access pool – brand<br>csic – brand<br>had5 – product<br>immunitybio – brand<br>kostaive – product<br>medigen – brand<br>mnexspike – product<br>moderna – brand<br>novavax – brand<br>nuvaxovid – product<br>oxford – brand<br>pfizer – brand<br>sanofi – brand<br>serum institute of india – brand<br>shanghai zerun biotechnology – brand<br>spikevax – product<br>vinbiocare – brand<br>walvax biotech – brand<br>who – brand<br>zapomeran – product | 202‑cov – product<br>arcturus therapeutics – brand<br>cansino biologics – brand<br>comirnaty – product<br>convidecia – product<br>hAd5 – product<br>immunitybio – brand<br>mNEXSPIKE – product<br>moderna – brand<br>novavax – brand<br>nuvaxovid – product<br>pfizer/biontech – brand<br>shanghai zerun biotechnology – brand<br>spikevax – product<br>vinbiocare – brand<br>walvax biotech – brand<br>zapomeran – product |
| 5 | Here’s a comprehensive overview of the most popular electric vehicle (EV) models in 2023, drawing from both global and U.S.-specific data. The landscape shows both Tesla’s continued dominance and the growing presence of Chinese manufacturers.<br><br>## Global Market Overview<br><br>- Tesla Model Y emerged as the top-selling EV globally in 2023, with approximately 772,364 units sold between January and August ([visualcapitalist.com](https://www.visualcapitalist.com/electric-vehicle-sales-by-model-2023/?utm_source=openai)). According to Tesla’s own data, total annual sales reached roughly 1.212 million units worldwide ([en.wikipedia.org](https://en.wikipedia.org/wiki/Tesla_Model_Y?utm_source=openai)).<br>- The Tesla Model 3 secured the second spot with 364,403 units sold in the same January-to-August period ([visualcapitalist.com](https://www.visualcapitalist.com/electric-vehicle-sales-by-model-2023/?utm_source=openai)).<br>- Chinese automakers featured prominently among the top sellers:<br>  - BYD Atto 3 (Yuan Plus) with 265,688 units ([visualcapitalist.com](https://www.visualcapitalist.com/electric-vehicle-sales-by-model-2023/?utm_source=openai)).<br>  - BYD Dolphin: 222,825 units ([visualcapitalist.com](https://www.visualcapitalist.com/electric-vehicle-sales-by-model-2023/?utm_source=openai)).<br>  - GAC Aion S: 160,693 units ([visualcapitalist.com](https://www.visualcapitalist.com/electric-vehicle-sales-by-model-2023/?utm_source=openai)).<br>  - Wuling HongGuang Mini EV: 153,399 units ([visualcapitalist.com](https://www.visualcapitalist.com/electric-vehicle-sales-by-model-2023/?utm_source=openai)).<br>  - GAC Aion Y: 136,619 units ([visualcapitalist.com](https://www.visualcapitalist.com/electric-vehicle-sales-by-model-2023/?utm_source=openai)).<br>  - BYD Seagull: 95,202 units ([evify.eu](https://www.evify.eu/articles/best-selling-ev-2023/?utm_source=openai)).<br>- The Volkswagen ID.4 also made the global top list with 120,154 units sold ([visualcapitalist.com](https://www.visualcapitalist.com/electric-vehicle-sales-by-model-2023/?utm_source=openai)).<br><br>These figures underscore Tesla’s international leadership and the rising significance of Chinese EV manufacturers. The BYD models, for instance, are gaining traction across Europe and Asia ([visualcapitalist.com](https://www.visualcapitalist.com/electric-vehicle-sales-by-model-2023/?utm_source=openai)).<br><br>## U.S. Market Breakdown<br><br>In the United States, Tesla continued to lead the EV market:<br><br>- Tesla Model Y: Around 394,497 units sold in 2023, making it the number-one EV in the U.S. ([electrek.co](https://electrek.co/2024/01/10/best-selling-evs-2023/?utm_source=openai)).<br>- Tesla Model 3: Approximately 220,910 units sold, securing the second spot ([electrek.co](https://electrek.co/2024/01/10/best-selling-evs-2023/?utm_source=openai)).<br>- Tesla Model X ranked ninth with about 23,015 units sold ([teslarati.com](https://www.teslarati.com/tesla-top-10-best-selling-evs-us-2023/?utm_source=openai)).<br>- Tesla accounted for roughly 55% of all EV sales in the U.S. in 2023 ([electrek.co](https://electrek.co/2024/01/10/best-selling-evs-2023/?utm_source=openai)).<br><br>Other top-selling EVs in the U.S. included:<br><br>1. Chevrolet Bolt EV/EUV: ~62,045 units ([electrek.co](https://electrek.co/2024/01/10/best-selling-evs-2023/?utm_source=openai)).<br>2. Ford Mustang Mach‑E: ~40,771 units ([electrek.co](https://electrek.co/2024/01/10/best-selling-evs-2023/?utm_source=openai)).<br>3. Volkswagen ID.4: ~37,789 units ([electrek.co](https://electrek.co/2024/01/10/best-selling-evs-2023/?utm_source=openai)).<br>4. Hyundai IONIQ 5: ~33,918 units ([electrek.co](https://electrek.co/2024/01/10/best-selling-evs-2023/?utm_source=openai)).<br>5. Rivian R1S: ~24,783 units ([electrek.co](https://electrek.co/2024/01/10/best-selling-evs-2023/?utm_source=openai)).<br>6. Ford F‑150 Lightning: ~24,165 units ([electrek.co](https://electrek.co/2024/01/10/best-selling-evs-2023/?utm_source=openai)).<br>7. BMW i4: ~22,583 units ([insideevs.com](https://insideevs.com/news/704178/20-best-selling-evs-us-2023/?utm_source=openai)).<br><br>Additional sales specifics include:<br><br>- The Ford F‑150 Lightning’s full-year sales were about 24,165 units, as confirmed by Ford’s internal numbers ([en.wikipedia.org](https://en.wikipedia.org/wiki/Ford_F-150_Lightning?utm_source=openai)).<br>- Combined sales of Rivian’s R1T and R1S reportedly reached ~47,200 units in 2023 ([caranddriver.com](https://www.caranddriver.com/news/g46351238/bestselling-ev-2023/?utm_source=openai))—though Kelley Blue Book lists R1S alone at 24,783 units ([electrek.co](https://electrek.co/2024/01/10/best-selling-evs-2023/?utm_source=openai)).<br><br>## Summary: Most Popular EV Models in 2023<br><br>Global top models:<br><br>- Tesla Model Y (~1.2 million worldwide)<br>- Tesla Model 3 (~364,000 Jan–Aug)<br>- BYD Atto 3/Yuan Plus, BYD Dolphin, GAC Aion S, Wuling HongGuang Mini EV, GAC Aion Y, BYD Seagull, VW ID.4<br><br>U.S. top models:<br><br>- Tesla Model Y (~394,500)<br>- Tesla Model 3 (~220,900)<br>- Chevrolet Bolt EV/EUV<br>- Ford Mustang Mach‑E<br>- Volkswagen ID.4<br>- Hyundai IONIQ 5<br>- Rivian R1S<br>- Ford F‑150 Lightning<br>- BMW i4<br><br>---<br><br>Let me know if you’d like a deeper dive into regional trends, EV market share by country, or projections based on this data. | bmw – brand<br>bmw i4 – product<br>byd – brand<br>byd atto 3 – product<br>byd dolphin – product<br>byd seagull – product<br>chevrolet – brand<br>chevrolet bolt ev/euv – product<br>ford – brand<br>ford f‑150 lightning – product<br>ford mustang mach‑e – product<br>gac – brand<br>gac aion s – product<br>gac aion y – product<br>hyundai – brand<br>hyundai ioniq 5 – product<br>rivian – brand<br>rivian r1s – product<br>rivian r1t – product<br>tesla – brand<br>tesla model 3 – product<br>tesla model y – product<br>volkswagen – brand<br>volkswagen id.4 – product<br>wuling – brand<br>wuling hongguang mini ev – product<br>yuan plus – product | bmw – brand<br>bmw i4 – product<br>byd – brand<br>byd atto 3 – product<br>byd dolphin – product<br>byd seagull – product<br>chevrolet – brand<br>chevrolet bolt euv – product<br>chevrolet bolt ev – product<br>ford – brand<br>ford f-150 lightning – product<br>ford mustang mach-e – product<br>gac – brand<br>gac aion s – product<br>gac aion y – product<br>hyundai – brand<br>hyundai ioniq 5 – product<br>rivian – brand<br>rivian r1s – product<br>rivian r1t – product<br>tesla – brand<br>tesla model 3 – product<br>tesla model x – product<br>tesla model y – product<br>volkswagen – brand<br>volkswagen id.4 – product<br>wuling – brand<br>wuling hongguang mini ev – product<br>yuan plus – product | aion s – product<br>aion y – product<br>atto 3 – product<br>bmw – brand<br>bolt ev/euv – product<br>byd – brand<br>chevrolet – brand<br>dolphin – product<br>f‑150 lightning – product<br>ford – brand<br>gac – brand<br>hongguang mini ev – product<br>hyundai – brand<br>i4 – product<br>id.4 – product<br>ioniq 5 – product<br>model 3 – product<br>model y – product<br>mustang mach‑e – product<br>r1s – product<br>rivian – brand<br>seagull – product<br>tesla – brand<br>volkswagen – brand<br>wuling – brand<br>yuan plus – product | aion s – product<br>aion y – product<br>atto 3 – product<br>bmw – brand<br>bolt euv – product<br>bolt ev – product<br>byd – brand<br>chevrolet – brand<br>dolphin – product<br>f-150 lightning – product<br>ford – brand<br>gac – brand<br>hongguang mini ev – product<br>hyundai – brand<br>i4 – product<br>id.4 – product<br>ioniq 5 – product<br>model 3 – product<br>model x – product<br>model y – product<br>mustang mach-e – product<br>r1s – product<br>r1t – product<br>rivian – brand<br>seagull – product<br>tesla – brand<br>volkswagen – brand<br>wuling – brand<br>yuan plus – product | aion s – product<br>aion y – product<br>atto 3 – product<br>bmw – brand<br>bolt euv – product<br>bolt ev – product<br>byd – brand<br>chevrolet – brand<br>dolphin – product<br>f-150 lightning – product<br>ford – brand<br>gac – brand<br>hongguang mini ev – product<br>hyundai – brand<br>i4 – product<br>id.4 – product<br>ioniq 5 – product<br>model 3 – product<br>model y – product<br>mustang mach-e – product<br>r1s – product<br>r1t – product<br>rivian – brand<br>seagull – product<br>tesla – brand<br>volkswagen – brand<br>wuling – brand<br>yuan plus – product | aion s – product<br>aion y – product<br>atto 3 – product<br>bolt ev – product<br>bwm – brand<br>byd – brand<br>chevrolet – brand<br>dolphin – product<br>euv – product<br>f-150 lightning – product<br>ford – brand<br>gac – brand<br>hongguang mini ev – product<br>hyundai – brand<br>i4 – product<br>id.4 – product<br>ioniq 5 – product<br>model 3 – product<br>model y – product<br>mustang mach-e – product<br>r1s – product<br>rivian – brand<br>seagull – product<br>tesla – brand<br>volkswagen – brand |
| 6 | Here are several fashion brands widely recognized for their sustainable practices as of late 2025. These selections represent a variety of approaches—including environmental innovation, ethical labor, circular economy efforts, transparency, and activism.<br><br>**Patagonia**  <br>• Long-established leader in sustainable and ethical fashion, especially in the outdoor sector. Known for durable garments, recycled materials, advocacy, and the "Worn Wear" repair and resale program encouraging reuse over new production. ([rankred.com](https://www.rankred.com/16-sustainable-fashion-brands-as-of-2025/?utm_source=openai))<br><br>**Stella McCartney**  <br>• Pioneering luxury designer avoiding leather, fur, and feathers. Embraces innovative plant- and bio-based materials—such as mushroom leather, algae-based Kelsun yarn, grape leather, and biodegradable rubber. Strong reductions in CO₂, water use, and waste; targets net-zero emissions by 2040. ([rankred.com](https://www.rankred.com/16-sustainable-fashion-brands-as-of-2025/?utm_source=openai))<br><br>**Reformation**  <br>• Trendy womenswear label using deadstock and recycled fabrics, low-impact dyes. Carbon-neutral operations, transparent supply chain, and impact data provided for purchases. ([reviewcollections.com](https://www.reviewcollections.com/blog/top-10-sustainable-fashion-brands-to-watch-in-2025/?utm_source=openai))<br><br>**Everlane**  <br>• Known for “radical transparency,” revealing cost breakdowns, factory conditions, and production processes. Uses sustainable materials like TENCEL™, recycled cotton, and certified wool. ([rankrumble.blogspot.com](https://rankrumble.blogspot.com/2025/03/top-10-sustainable-fashion-brands-to.html?utm_source=openai))<br><br>**Allbirds**  <br>• Eco-friendly footwear and apparel brand using natural materials (eucalyptus fiber, merino wool, sugarcane). Commits to carbon-neutral production and biodegradable designs. ([rankrumble.blogspot.com](https://rankrumble.blogspot.com/2025/03/top-10-sustainable-fashion-brands-to.html?utm_source=openai))<br><br>**PANGAIA**  <br>• Science-driven brand combining sustainable materials—like seaweed fibers, eucalyptus pulp, recycled plastic, grape waste—with modern streetwear. Appeals strongly to Gen Z. ([9trendz.com](https://www.9trendz.com/2025/08/top-10-sustainable-fashion-brands-that.html?utm_source=openai))<br><br>**People Tree**  <br>• Pioneering fair-trade fashion brand offering organic cotton, hemp, and handcrafted designs. Well-known for ethical production and artisan support. ([rankrumble.blogspot.com](https://rankrumble.blogspot.com/2025/03/top-10-sustainable-fashion-brands-to.html?utm_source=openai))<br><br>**Tentree**  <br>• Eco-conscious Canadian brand that plants ten trees for every item sold. Uses organic cotton, hemp, Tencel; certified ethical manufacturing. Over 100 million trees planted globally to date. ([reviewcollections.com](https://www.reviewcollections.com/blog/top-10-sustainable-fashion-brands-to-watch-in-2025/?utm_source=openai))<br><br>**Toad & Co**  <br>• Outdoor lifestyle label focused on slow fashion, recycled and natural fibers, fair-trade manufacturing. Offers a pre-loved resale program to reduce waste. ([reviewcollections.com](https://www.reviewcollections.com/blog/top-10-sustainable-fashion-brands-to-watch-in-2025/?utm_source=openai))<br><br>**Warp + Weft**  <br>• Size-inclusive denim brand with transparent manufacturing. Uses minimal water (under 10 gallons per pair), recycles 98% of water, and provides full supply chain visibility. ([reviewcollections.com](https://www.reviewcollections.com/blog/top-10-sustainable-fashion-brands-to-watch-in-2025/?utm_source=openai))<br><br>**MagicLinen**  <br>• Lithuanian linen specialist producing minimal-waste, biodegradable garments in small batches using only linen and eco-friendly packaging. ([reviewcollections.com](https://www.reviewcollections.com/blog/top-10-sustainable-fashion-brands-to-watch-in-2025/?utm_source=openai))<br><br>**Gucci (Off the Grid & other initiatives)**  <br>• Luxury brand implementing sustainability through its “Equilibrium” platform. Launched the “Off the Grid” collection using recycled/organic materials; invested in circular hubs and animal-free materials (e.g., Demetra); recognized with circular economy awards. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gucci?utm_source=openai))<br><br>**Eileen Fisher**  <br>• Elevated eco-luxury label focusing on timeless design, recycling, and waste reduction through its “Renew” take-back program. Committed to regenerative agriculture and sustainable dyeing. ([dxbnewsnetwork.com](https://www.dxbnewsnetwork.com/top-10-sustainable-luxury-fashion-brands-to-watch-in-2025?utm_source=openai))<br><br>**Hermès (Sylvania Project)**  <br>• Iconic luxury house exploring sustainable materials via “Sylvania,” including mycelium-based vegan leather. Represents high-craft sustainable innovation. ([dxbnewsnetwork.com](https://www.dxbnewsnetwork.com/top-10-sustainable-luxury-fashion-brands-to-watch-in-2025?utm_source=openai))<br><br>**Akyn (by Amy Powney)**  <br>• Launched in 2025 by designer Amy Powney (formerly of Mother of Pearl). Emphasizes transparency, small-scale production, organic fibers, and meaningful craftsmanship. Designed as “affordable luxury,” with sustainability at its core. ([theguardian.com](https://www.theguardian.com/fashion/2025/may/26/greenwashing-is-rife-is-amy-powney-the-most-sustainable-designer-in-fashion?utm_source=openai))<br><br>**Other Notable Mentions**  <br>• **Mother of Pearl**: Known for B Corp status, use of organic materials, and ethical production. Amy Powney’s leadership significantly raised its sustainable profile. ([marieclaire.co.uk](https://www.marieclaire.co.uk/news/fashion-news/best-ethical-fashion-brands-to-know-sustainable-fashion-84169?utm_source=openai))  <br>• **Rise & Fall, Sézane, Nobody's Child**: B Corp-certified labels highlighted in September 2025 coverage for their high-quality, sustainable materials, transparent processes, and philanthropic engagement. ([marieclaire.co.uk](https://www.marieclaire.co.uk/news/fashion-news/best-ethical-fashion-brands-to-know-sustainable-fashion-84169?utm_source=openai))<br><br>---<br><br>Summary  <br>These brands span a wide spectrum—from outdoor apparel and streetwear to high luxury—yet share a commitment to sustainable materials, transparency, ethical labor, circularity, or environmental activism. Whether you're seeking mainstream basics (Everlane), bold science-infused fashion (PANGAIA), luxury with conscience (Stella McCartney, Gucci, Hermès), or community-rooted small-scale design (Akyn, People Tree), there are credible sustainable options available across fashion categories.<br><br>Let me know if you'd like deeper profiles, comparisons, or suggestions tailored to specific styles or budgets! | akyn – brand<br>allbirds – brand<br>demetra – product<br>eileen fisher – brand<br>equilibrium – product<br>everlane – brand<br>gucci – brand<br>hermès – brand<br>magiclinen – brand<br>mother of pearl – brand<br>nobody's child – brand<br>off the grid – product<br>pangaia – brand<br>patagonia – brand<br>people tree – brand<br>reformation – brand<br>renew – product<br>rise & fall – brand<br>sézane – brand<br>stella mccartney – brand<br>sylviana – product<br>tencel™ – product<br>tentree – brand<br>toad & co – brand<br>warp + weft – brand<br>worn wear – product | akyn – brand<br>allbirds – brand<br>demetra – product<br>eileen fisher – brand<br>equilibrium – product<br>everlane – brand<br>gucci – brand<br>hermès – brand<br>kelsun yarn – product<br>magiclinen – brand<br>mother of pearl – brand<br>nobody's child – brand<br>off the grid – product<br>pangaia – brand<br>patagonia – brand<br>people tree – brand<br>reformation – brand<br>renew – product<br>rise & fall – brand<br>sézane – brand<br>stella mccartney – brand<br>sylvania project – product<br>tencel – product<br>tentree – brand<br>toad & co – brand<br>warp + weft – brand<br>worn wear – product | akyn – brand<br>allbirds – brand<br>eileen fisher – brand<br>equilibrium – product<br>everlane – brand<br>gucci – brand<br>hermes – brand<br>magiclinen – brand<br>mother of pearl – brand<br>nobody's child – brand<br>off the grid – product<br>pangaia – brand<br>patagonia – brand<br>people tree – brand<br>reformation – brand<br>renew – product<br>rise & fall – brand<br>sézane – brand<br>stella mccartney – brand<br>sylvania – product<br>tentree – brand<br>toad & co – brand<br>warp + weft – brand<br>worn wear – product | akyn – brand<br>allbirds – brand<br>demetra – product<br>eileen fisher – brand<br>equilibrium – product<br>everlane – brand<br>gucci – brand<br>hermès – brand<br>kelsun yarn – product<br>magiclinen – brand<br>mother of pearl – brand<br>nobody's child – brand<br>off the grid – product<br>pangaia – brand<br>patagonia – brand<br>people tree – brand<br>reformation – brand<br>renew – product<br>rise & fall – brand<br>sézane – brand<br>stella mccartney – brand<br>sylvania – product<br>tencel – product<br>tentree – brand<br>toad & co – brand<br>warp + weft – brand<br>worn wear – product | akyn – brand<br>allbirds – brand<br>eileen fisher – brand<br>everlane – brand<br>gucci – brand<br>hermès – brand<br>magiclinen – brand<br>mother of pearl – brand<br>nobody's child – brand<br>off the grid – product<br>pangaia – brand<br>patagonia – brand<br>people tree – brand<br>reformation – brand<br>rise & fall – brand<br>sézane – brand<br>stella mccartney – brand<br>sylvania – product<br>tencel – product<br>tentree – brand<br>toad & co – brand<br>warp + weft – brand<br>worn wear – product | akyn – brand<br>allbirds – brand<br>eileen fisher – brand<br>everlane – brand<br>gucci – brand<br>hermès – brand<br>magicl linen – brand<br>mother of pearl – brand<br>pangaia – brand<br>patagonia – brand<br>people tree – brand<br>reformation – brand<br>stella mccartney – brand<br>tentree – brand<br>toad & co – brand<br>warp + weft – brand |
| 7 | Here’s an updated look at the **top social media platforms by user base** as of late 2025, based on the most recent figures available:<br><br>---<br><br>##  Leading Platforms by Monthly Active Users (MAUs)<br><br>- **Facebook** remains the most widely used platform, with approximately **3.07 billion monthly active users**. ([broadbandsearch.net](https://www.broadbandsearch.net/blog/social-media-facts-statistics?utm_source=openai))  <br>- **WhatsApp** ranks closely behind, also with around **3 billion users**, primarily in regions that rely heavily on messaging apps. ([quantumrun.com](https://www.quantumrun.com/consulting/most-popular-social-media/?utm_source=openai))  <br>- **YouTube** reaches between **2.5 to 2.7 billion users**, making it the leading video-sharing platform globally. ([broadbandsearch.net](https://www.broadbandsearch.net/blog/social-media-facts-statistics?utm_source=openai))  <br>- **Instagram** has seen substantial growth, now reaching **2 billion to possibly up to 3 billion monthly users** according to Meta CEO’s recent announcement in September 2025. ([reuters.com](https://www.reuters.com/business/meta-ceo-zuckerberg-says-instagram-has-grown-3-billion-monthly-active-users-2025-09-24/?utm_source=openai))  <br>- **TikTok** continues its rapid ascent with roughly **1.8 to 1.94 billion MAUs**, firmly establishing itself as a short-form video powerhouse. ([medium.com](https://medium.com/%40contact.resourcera/social-media-statistics-d0ab4b217250?utm_source=openai))  <br><br>Other platforms with significant user counts include:<br>- **WeChat**: ~1.3–1.4 billion users, with dominant use in China. ([demandsage.com](https://www.demandsage.com/social-media-users/?utm_source=openai))  <br>- **Telegram**: approximately **1 billion users**, gaining traction thanks to its privacy features. ([medium.com](https://medium.com/%40contact.resourcera/social-media-statistics-d0ab4b217250?utm_source=openai))  <br>- **Messenger (Facebook Messenger)**: around **950 million to 979 million users**. ([broadbandsearch.net](https://www.broadbandsearch.net/blog/social-media-facts-statistics?utm_source=openai))  <br>- **Snapchat**: nearing **900 million users**, particularly popular among younger demographics. ([lifesight.io](https://lifesight.io/blog/popular-social-media-platforms/?utm_source=openai))  <br>- **Reddit**: estimates range between **1.1 to 1.36 billion monthly unique visitors**. ([boxwoodco.com](https://www.boxwoodco.com/articles/top-social-media-platforms-2025?utm_source=openai))  <br><br>---<br><br>##  Summary Table: 2025 Estimates by Monthly Users<br><br>- Facebook: ~3.07B  <br>- WhatsApp: ~3.00B  <br>- YouTube: ~2.5–2.7B  <br>- Instagram: ~2.0B (up to 3B per recent announcements)  <br>- TikTok: ~1.8–1.94B  <br>- WeChat: ~1.3–1.4B  <br>- Telegram: ~1.0B  <br>- Messenger: ~950M–979M  <br>- Snapchat: ~900M  <br>- Reddit: ~1.1–1.36B  <br><br>---<br><br>##  Context & Insights<br><br>Platforms vary not only in scale but also in function and regional dominance. Facebook and WhatsApp lead in sheer numbers; YouTube dominates in video consumption; Instagram leverages visual storytelling and influencer marketing; TikTok continues rapid expansion among younger audiences. Messaging apps like WeChat and Telegram remain essential communication tools in their respective spheres.<br><br>Instagram’s milestone of reaching **3 billion MAUs**, announced by Meta CEO Mark Zuckerberg in September 2025, reflects its significant growth trajectory, likely influenced by features like Reels. ([reuters.com](https://www.reuters.com/business/meta-ceo-zuckerberg-says-instagram-has-grown-3-billion-monthly-active-users-2025-09-24/?utm_source=openai))<br><br>---<br><br>In short, as of late 2025, the social media landscape is dominated by Meta-owned platforms (Facebook, WhatsApp, Instagram, Messenger), with YouTube and TikTok trailing closely behind, and regional leaders like WeChat and Telegram also holding substantial user bases.<br><br>Feel free to ask if you’d like further breakdowns by region, demographics, or daily active usage! | facebook – brand<br>facebook – product<br>instagram – brand<br>instagram – product<br>messenger – brand<br>messenger – product<br>reddit – brand<br>reddit – product<br>snapchat – brand<br>snapchat – product<br>telegram – brand<br>telegram – product<br>tiktok – brand<br>tiktok – product<br>wechat – brand<br>wechat – product<br>whatsapp – brand<br>whatsapp – product<br>youtube – brand<br>youtube – product | facebook – brand<br>instagram – brand<br>messenger – brand<br>meta – brand<br>reddit – brand<br>reels – product<br>snapchat – brand<br>telegram – brand<br>tiktok – brand<br>wechat – brand<br>whatsapp – brand<br>youtube – brand | facebook – brand<br>instagram – brand<br>messenger – brand<br>reddit – brand<br>snapchat – brand<br>telegram – brand<br>tiktok – brand<br>wechat – brand<br>whatsapp – brand<br>youtube – brand | facebook – brand<br>instagram – brand<br>messenger – brand<br>reddit – brand<br>snapchat – brand<br>telegram – brand<br>tiktok – brand<br>wechat – brand<br>whatsapp – brand<br>youtube – brand | facebook – brand<br>instagram – brand<br>messenger – product<br>reddit – brand<br>snapchat – brand<br>telegram – brand<br>tiktok – brand<br>wechat – brand<br>whatsapp – brand<br>youtube – brand | facebook – brand<br>instagram – brand<br>messenger – brand<br>reddit – brand<br>snapchat – brand<br>telegram – brand<br>tiktok – brand<br>wechat – brand<br>whatsapp – brand<br>youtube – brand |
| 8 | Here’s a structured overview of which airlines currently offer the most outstanding **in-flight services**—drawing from recent 2025 awards, expert analyses, and trusted travel publications:<br><br>---<br><br>##  Global Leaders in In‑Flight Service<br><br>**1. Qatar Airways**  <br>- Crowned **World’s Best Airline** for 2025 by Skytrax for the second year running, recognized for its exceptional global service and premium cabins.([en.as.com](https://en.as.com/latest_news/the-best-in-flight-experience-in-the-world-these-are-the-top-5-airlines-in-2025-according-to-skytrax-n/?utm_source=openai))  <br>- Forbes Travel Guide named its **Qsuites** the **Best International Business Class**, and Emirates’ first-class beat them only in the first-class category.([forbes.com](https://www.forbes.com/sites/forbestravelguide/2025/10/20/the-worlds-best-airlines-airports-2025-forbes-travel-guide-verified-air-travel-awards/?utm_source=openai))  <br><br>**2. Singapore Airlines**  <br>- Regularly praised for its premium service and luxury touches.([outdoorkeeda.com](https://www.outdoorkeeda.com/news/top-10-best-airlines-in-the-world-2025-rankings-reviews-and-travel-experience?utm_source=openai))  <br>- Skytrax ranks it among the top global airlines—often placing second.([en.as.com](https://en.as.com/latest_news/the-best-in-flight-experience-in-the-world-these-are-the-top-5-airlines-in-2025-according-to-skytrax-n/?utm_source=openai))  <br><br>**3. Cathay Pacific**  <br>- Excelled in long-haul comfort and entertainment offerings.([outdoorkeeda.com](https://www.outdoorkeeda.com/news/top-10-best-airlines-in-the-world-2025-rankings-reviews-and-travel-experience?utm_source=openai))  <br>- Awarded **World’s Best In-Flight Entertainment** in 2025 by Skytrax, ahead of Emirates and Qatar.([worldairlineawards.com](https://www.worldairlineawards.com/worlds-best-inflight-entertainment-2024/?utm_source=openai))  <br><br>**4. Emirates**  <br>- Continues to lead in luxury and entertainment, especially with its ICE system.([news.com.au](https://www.news.com.au/travel/travel-advice/flights/emirates-is-best-airline-in-the-world-for-this/news-story/bdd2eb644679fb716dca4c91ecab1684?utm_source=openai))  <br>- Forbes named it **Best International Airline**, and also awarded its **First Class suites** the top spot.([forbes.com](https://www.forbes.com/sites/forbestravelguide/2025/10/20/the-worlds-best-airlines-airports-2025-forbes-travel-guide-verified-air-travel-awards/?utm_source=openai))  <br><br>**5. Air New Zealand & Korean Air**  <br>- Air New Zealand: Awarded **world’s safest airline** by AirlineRatings for 2025.([nypost.com](https://nypost.com/2025/01/16/lifestyle/worlds-safest-airlines-revealed-for-2025-alaska-is-top-us-carrier/?utm_source=openai))  <br>- Korean Air named **World’s Best Airline** by AirlineRatings for its passenger comfort—from roomy economy to thoughtful touches like slippers and multi-continental menus.([the-independent.com](https://www.the-independent.com/travel/news-and-advice/best-airline-world-2025-b2700139.html?utm_source=openai))  <br><br>---<br><br>##  Best In-Flight Service by Category<br><br>\| Cabin / Category              \| Top Airline(s) & Highlights \|<br>\|------------------------------\|------------------------------\|<br>\| Economy                      \| Korean Air – superior seat pitch (31+ inches), complimentary slippers, generous meals([the-independent.com](https://www.the-independent.com/travel/news-and-advice/best-airline-world-2025-b2700139.html?utm_source=openai)) \|<br>\| Premium Economy              \| Tie between Qantas and Emirates; Emirates' is notably luxurious([forbes.com](https://www.forbes.com/sites/laurabegleybloom/2025/02/10/the-best-airlines-in-2025-according-to-a-new-travel-ratings-report/?utm_source=openai)) \|<br>\| Business Class               \| Qatar Airways (Qsuites) – private suites, double-bed option([forbes.com](https://www.forbes.com/sites/laurabegleybloom/2025/02/10/the-best-airlines-in-2025-according-to-a-new-travel-ratings-report/?utm_source=openai)) \|<br>\| First Class                  \| Singapore Airlines (Skyroom Suites) and Emirates (First Class suites with Bulgari, chauffeurs)([forbes.com](https://www.forbes.com/sites/laurabegleybloom/2025/02/10/the-best-airlines-in-2025-according-to-a-new-travel-ratings-report/?utm_source=openai)) \|<br>\| In-Flight Entertainment      \| Cathay Pacific – best overall; Emirates close behind([worldairlineawards.com](https://www.worldairlineawards.com/worlds-best-inflight-entertainment-2024/?utm_source=openai)) \|<br><br>---<br><br>##  North American Carriers – Standout In-Flight Features<br><br>**Delta Air Lines**  <br>- Named **Best U.S. Airline** by Forbes Travel Guide in 2025. Known for premium food/beverage partners (e.g., Shake Shack, Champagne Taittinger), strong customer service, and complimentary Wi-Fi.([forbes.com](https://www.forbes.com/sites/forbestravelguide/2025/10/20/the-worlds-best-airlines-airports-2025-forbes-travel-guide-verified-air-travel-awards/?utm_source=openai))  <br><br>**JetBlue**  <br>- Ranked most broadly as the top **hybrid airline**, offering free Wi-Fi, live TV, snacks, spacious seats (32–33"), and Mint class suites.([forbes.com](https://www.forbes.com/sites/laurabegleybloom/2025/02/10/the-best-airlines-in-2025-according-to-a-new-travel-ratings-report/?utm_source=openai))  <br><br>**Delta, JetBlue, Southwest, and United**  <br>- According to NerdWallet, these airlines lead in in-flight experience among U.S. carriers. Notably, only JetBlue and Delta offer consistently **free Wi-Fi and messaging** across most flights.([nerdwallet.com](https://www.nerdwallet.com/article/travel/best-airline-entertainment?utm_source=openai))  <br><br>---<br><br>##  Summary: Best Airlines by Service Focus<br><br>- **Ultimate global service**: Qatar Airways, Singapore Airlines, Emirates, Cathay Pacific  <br>- **Best in economy comfort**: Korean Air  <br>- **Best premium economy and first-class luxury**: Emirates, Singapore Airlines  <br>- **Best in entertainment**: Cathay Pacific, Emirates  <br>- **Top safety**: Air New Zealand  <br>- **Best U.S. in-flight offerings**: Delta (overall), JetBlue (hybrid model)  <br><br>---<br><br>##  Final Takeaway<br><br>If **luxury, comfort, and top-tier service** are your priorities on long-haul or international flights, **Qatar Airways**, **Singapore Airlines**, **Emirates**, and **Cathay Pacific** stand out as leaders in 2025. For **value-driven comfort**, especially in economy, **Korean Air** delivers impressively. In the **U.S. market**, **Delta** and **JetBlue** offer superior in-flight amenities, while **Air New Zealand** is a standout for safety.<br><br>Let me know if you'd like recommendations tailored to specific routes, cabin classes, or loyalty programs! | air new zealand – brand<br>bulgari – brand<br>cathay pacific – brand<br>champagne taittinger – brand<br>delta – brand<br>delta air lines – brand<br>emirates – brand<br>first class suites – product<br>ice – product<br>jetblue – brand<br>korean air – brand<br>mint class suites – product<br>qantas – brand<br>qatar airways – brand<br>qsuites – product<br>shake shack – brand<br>singapore airlines – brand<br>skyroom suites – product<br>southwest – brand<br>united – brand | air new zealand – brand<br>airlineratings – brand<br>bulgari – brand<br>cathay pacific – brand<br>delta air lines – brand<br>emirates – brand<br>forbes travel guide – brand<br>ice system – product<br>jetblue – brand<br>korean air – brand<br>mint class suite – product<br>nerdwallet – brand<br>qantas – brand<br>qatar airways – brand<br>qsuite – product<br>shake shack – brand<br>singapore airlines – brand<br>skyroom suite – product<br>skytrax – brand<br>southwest – brand<br>taittinger – brand<br>united – brand<br>world airline awards – brand | air new zealand – brand<br>cathay pacific – brand<br>delta air lines – brand<br>emirates – brand<br>first class suites – product<br>ice system – product<br>jetblue – brand<br>korean air – brand<br>mint class suites – product<br>qatar airways – brand<br>qsuites – product<br>singapore airlines – brand<br>skyroom suites – product<br>southwest – brand<br>united – brand | air new zealand – brand<br>business class – product<br>cathay pacific – brand<br>delta – brand<br>economy – product<br>emirates – brand<br>first class – product<br>first class suite – product<br>free wi-fi and messaging – product<br>ice – product<br>in-flight entertainment – product<br>jetblue – brand<br>korean air – brand<br>mint class suite – product<br>premium economy – product<br>qatar airways – brand<br>qsuite – product<br>singapore airlines – brand<br>skyroom suite – product<br>southwest – brand<br>united – brand<br>wi-fi – product | air new zealand – brand<br>bulgari – brand<br>cathay pacific – brand<br>champagne taittinger – brand<br>chauffeur – product<br>delta air line – brand<br>emirates – brand<br>first class suite – product<br>ice system – product<br>jetblue – brand<br>korean air – brand<br>mint suite – product<br>qantas – brand<br>qatar airways – brand<br>qsuite – product<br>shake shack – brand<br>singapore airline – brand<br>skyroom suite – product<br>southwest airline – brand<br>united airline – brand<br>wifi – product | air new zealand – brand<br>cathay pacific – brand<br>emirates – brand<br>first class suites – product<br>ice system – product<br>korean air – brand<br>qatar airways – brand<br>qsuies – product<br>singapore airlines – brand<br>skyroom suites – product |
| 9 | Here’s a detailed and well-cited overview of the leading technology companies in artificial intelligence (AI) as of late 2025:<br><br>---<br><br>###  1. Nvidia  <br>- **Role**: Dominates the AI hardware sector with its GPUs, vital for AI model training and execution. Holds around a 92% share in the data center GPU market, and its AI-related revenue in 2025 is estimated to exceed $30 billion ([thetechedvocate.org](https://www.thetechedvocate.org/top-100-ai-companies-revolutionizing-technology-in-2025/?utm_source=openai)).  <br>- **Market Value**: Became the first company to reach a $4 trillion market valuation in mid‑2025 and later climbed to ~$4.5 trillion ([en.wikipedia.org](https://en.wikipedia.org/wiki/Nvidia?utm_source=openai)). Its explosive stock surge is emblematic of investor confidence in AI infrastructure ([businessinsider.com](https://www.businessinsider.com/ai-stocks-tech-nvidia-openai-coke-mcdonalds-microsoft-alphabet-oracle-2025-10?utm_source=openai)).<br><br>---<br><br>###  2. Microsoft  <br>- **AI Integration**: Embedded AI across its ecosystem—from Azure OpenAI services and Bing AI to Copilot in Windows, Office, GitHub, and Teams ([nyongesasande.com](https://www.nyongesasande.com/top-ai-companies-2025/?utm_source=openai)).  <br>- **Investment Commitment**: Announced an $80 billion investment in U.S.-based AI infrastructure in 2025 ([usfinanceinsider.com](https://usfinanceinsider.com/2025/03/26/global-ai-stocks-9-biggest-companies-in-2025/?utm_source=openai)).  <br>- **Market Impact**: Added about $800 billion in market value in 2025, benefiting from enterprise adoption and AI-driven growth ([businessinsider.com](https://www.businessinsider.com/ai-stocks-tech-nvidia-openai-coke-mcdonalds-microsoft-alphabet-oracle-2025-10?utm_source=openai)).<br><br>---<br><br>###  3. Alphabet (Google)—including DeepMind, Gemini, Bard  <br>- **Research & Product Reach**: DeepMind leads foundational AI research, while Gemini (formerly Bard) powers conversational AI across Google’s services ([nyongesasande.com](https://www.nyongesasande.com/top-ai-companies-2025/?utm_source=openai)).  <br>- **Innovation Investment**: Announced a massive $75 billion commitment to AI in 2025 ([thetechedvocate.org](https://www.thetechedvocate.org/top-100-ai-companies-revolutionizing-technology-in-2025/?utm_source=openai)).  <br>- **Patent Leadership**: As of early 2025, Google leads in U.S. generative AI and agentic AI patent filings—surpassing IBM—with Microsoft, Nvidia, and IBM trailing behind ([axios.com](https://www.axios.com/2025/05/15/ai-patents-google-agents?utm_source=openai)).  <br>- **Valuation Gains**: Saw a ~$740 billion addition in market value during 2025 ([businessinsider.com](https://www.businessinsider.com/ai-stocks-tech-nvidia-openai-coke-mcdonalds-microsoft-alphabet-oracle-2025-10?utm_source=openai)).<br><br>---<br><br>###  4. OpenAI  <br>- **Generative AI Pioneer**: Known for ChatGPT and the GPT series, OpenAI is central to AI’s mainstream rise ([skynetforums.com](https://skynetforums.com/top-10-u-s-ai-companies-shaping-the-future-in-2025/?utm_source=openai)).  <br>- **Financial Scale**: Estimated valuation around $300 billion, supported by a record-breaking $40 billion funding round ([remkokoster.nl](https://www.remkokoster.nl/top-30-ai-companies-major-acquisitions-2025/?utm_source=openai)).  <br>- **Strategic Partnerships**: Deeply integrated into Microsoft’s AI offerings—especially through Azure—helping drive enterprise adoption ([usfinanceinsider.com](https://usfinanceinsider.com/2025/03/26/global-ai-stocks-9-biggest-companies-in-2025/?utm_source=openai)).  <br>- **Backing from SoftBank**: SoftBank has committed up to $40 billion in new investment to OpenAI, having already invested $32 billion since late 2024 ([reuters.com](https://www.reuters.com/technology/softbank-aiming-become-leading-artificial-super-intelligence-platform-provider-2025-06-27/?utm_source=openai)).<br><br>---<br><br>###  5. Anthropic  <br>- **Specialization**: Focuses on aligned, safe, and steerable LLMs through its Claude model line ([nyongesasande.com](https://www.nyongesasande.com/top-ai-companies-2025/?utm_source=openai)).  <br>- **Funding & Valuation**: Completed a $3.5 billion Series E round, bringing its valuation to over $61 billion ([crn.com](https://www.crn.com/news/ai/2025/the-10-hottest-ai-startup-companies-of-2025-so-far?utm_source=openai)).<br><br>---<br><br>### Additional Notable Players<br><br>- **AI Infrastructure & Startups**  <br>  - *Anysphere (Cursor)*: Specializes in developer tools and code generation; has $100 million in annual contracts and $2.5 billion valuation ([forbes.com](https://www.forbes.com/lists/ai50/?utm_source=openai)).  <br>  - *Writer & Others*: Enterprise-focused AI model companies like Writer (marketing automation), Crusoe, Together AI, Lambda—key compute and model platform players ([forbes.com](https://www.forbes.com/lists/ai50/?utm_source=openai)).<br><br>- **China-based AI Leaders**  <br>  - *Moonshot AI*: Beijing-based startup known for LLMs like Kimi; valued at $3.3 billion following major funding and releasing long‑context models ([en.wikipedia.org](https://en.wikipedia.org/wiki/Moonshot_AI?utm_source=openai)).  <br>  - *Alibaba Cloud, ByteDance, Tencent, Zhipu*: Competitive AI model developers challenging Western firms on performance and cost ([theguardian.com](https://www.theguardian.com/technology/2025/feb/03/deepseek-chinese-ai-company-competitors-chatgpt?utm_source=openai)).<br><br>- **Infrastructure Acquisitions**  <br>  - *CoreWeave*: Emerging as a critical AI compute provider; has struck a $12 billion cloud contract with OpenAI, acquired Weights & Biases, and deployed Nvidia GB300 GPUs via cloud―went public in March 2025 ([en.wikipedia.org](https://en.wikipedia.org/wiki/CoreWeave?utm_source=openai)).<br><br>- **SoftBank’s Rising Influence**  <br>  SoftBank is strategically positioning itself in AI by acquiring Ampere and pledging sizable investments into OpenAI, aiming to become a leading "artificial super intelligence" platform provider within a decade ([reuters.com](https://www.reuters.com/technology/softbank-aiming-become-leading-artificial-super-intelligence-platform-provider-2025-06-27/?utm_source=openai)).<br><br>---<br><br>###  Summary Table<br><br>\| Company / Play \| Role in AI \|<br>\|----------------\|------------\|<br>\| Nvidia         \| Hardware backbone (GPUs) \|<br>\| Microsoft      \| Ecosystem-wide AI integration \|<br>\| Google/Alphabet\| Research leader and product integration \|<br>\| OpenAI         \| Generative AI pioneer \|<br>\| Anthropic      \| Ethical, aligned AI development \|<br>\| Anysphere, Writer, Crusoe, Lambda, Together \| Specialized AI infrastructure and tooling \|<br>\| Moonshot and Chinese firms \| Emerging model developers in Asia \|<br>\| CoreWeave      \| Infrastructure provider fueling AI compute \|<br>\| SoftBank       \| Investor and platform strategist \|<br><br>---<br><br>### Final Thoughts<br><br>These companies collectively define the landscape of AI in 2025 across infrastructure, foundational models, ethical AI, and investing. Nvidia and Microsoft have emerged as titans through hardware dominance and ecosystem integration, while Google maintains leadership in research and patenting. OpenAI and Anthropic represent the frontier of generative and safe AI development. A burgeoning group of infrastructure innovators and global players—including CoreWeave, Moonshot, and major Chinese firms—further diversify the space.<br><br>Let me know if you'd like to dive deeper into any specific company, technology, or regional trend!<br> | alibaba cloud – brand<br>alphabet – brand<br>ampere – brand<br>anthropic – brand<br>anysphere – brand<br>azure openai services – product<br>bard – product<br>bing ai – product<br>bytedance – brand<br>chatgpt – product<br>claude – product<br>copilot – product<br>coreweave – brand<br>crusoe – brand<br>cursor – product<br>deepmind – brand<br>gb300 – product<br>gemini – product<br>github – product<br>google – brand<br>gpt series – product<br>gpu – product<br>kimi – product<br>lambda – brand<br>microsoft – brand<br>moonshot ai – brand<br>nvidia – brand<br>office – product<br>openai – brand<br>softbank – brand<br>teams – product<br>tencent – brand<br>together ai – brand<br>weights & biases – brand<br>windows – product<br>writer – brand<br>zhipu – brand | alibaba cloud – brand<br>alphabet – brand<br>ampere – brand<br>anthropic – brand<br>anysphere – brand<br>azure openai – product<br>bard – product<br>bing ai – product<br>bytedance – brand<br>chatgpt – product<br>claude – product<br>copilot – product<br>coreweave – brand<br>crusoe – brand<br>cursor – product<br>deepmind – brand<br>gemini – product<br>github – product<br>google – brand<br>gpt – product<br>kimi – product<br>lambda – brand<br>microsoft – brand<br>moonshot ai – brand<br>nvidia – brand<br>nvidia gb300 gpu – product<br>office – product<br>openai – brand<br>softbank – brand<br>teams – product<br>tencent – brand<br>together ai – brand<br>weights & biases – brand<br>windows – product<br>writer – brand<br>zhipu – brand | alibaba cloud – brand<br>alphabet – brand<br>anthropic – brand<br>anysphere – brand<br>azure openai services – product<br>bard – product<br>bard – product<br>bytedance – brand<br>chatgpt – product<br>claude – product<br>copilot – product<br>coreweave – brand<br>crusoe – brand<br>gb300 gpu – product<br>gemini – product<br>github – product<br>gpt series – product<br>lambda – brand<br>microsoft – brand<br>moonshot ai – brand<br>nvidia – brand<br>office – product<br>openai – brand<br>softbank – brand<br>teams – product<br>tencent – brand<br>weights & biases – product<br>windows – product<br>writer – brand<br>zhipu – brand | alibaba cloud – brand<br>alphabet – brand<br>ampere – brand<br>anthropic – brand<br>anysphere – brand<br>azure openai service – product<br>bard – product<br>bing ai – product<br>bytedance – brand<br>chatgpt – product<br>claude – product<br>copilot – product<br>coreweave – brand<br>crusoe – brand<br>cursor – product<br>deepmind – brand<br>gb300 gpu – product<br>gemini – product<br>github – product<br>google – brand<br>gpt series – product<br>gpu – product<br>kimi – product<br>lambda – brand<br>microsoft – brand<br>moonshot ai – brand<br>nvidia – brand<br>office – product<br>openai – brand<br>softbank – brand<br>teams – product<br>tencent – brand<br>together ai – brand<br>weights & biases – product<br>windows – product<br>writer – brand<br>zhipu – brand | alibaba cloud – brand<br>alphabet – brand<br>ampere – brand<br>anthropic – brand<br>anysphere – brand<br>azure openai service – product<br>bard – product<br>bing ai – product<br>bytedance – brand<br>chatgpt – product<br>claude model – product<br>copilot – product<br>coreweave – brand<br>crusoe – brand<br>cursor – product<br>deepmind – product<br>gemini – product<br>github – product<br>google – brand<br>gpt series – product<br>gpu – product<br>kimi – product<br>lambda – brand<br>microsoft – brand<br>moonshot ai – brand<br>nvidia – brand<br>office – product<br>openai – brand<br>softbank – brand<br>teams – product<br>tencent – brand<br>together ai – brand<br>weights & biases – product<br>windows – product<br>writer – brand<br>zhipu – brand | alibaba cloud – brand<br>alphabet – brand<br>ampere – brand<br>anthropic – brand<br>anysphere – brand<br>bard – product<br>bytedance – brand<br>chatgpt – product<br>claude – product<br>coreweave – brand<br>crusoe – brand<br>cursor – product<br>deepmind – brand<br>gemini – product<br>google – brand<br>gpt – product<br>kimi – product<br>lambda – brand<br>microsoft – brand<br>moonshot ai – brand<br>nvidia – brand<br>openai – brand<br>openai – brand<br>tencent – brand<br>together ai – brand<br>weights & biases – product<br>writer – brand<br>zhipu – brand |
| 10 | Here’s a detailed and updated overview of food brands known for having healthier product lines in the U.S. market as of late 2025, along with supporting citations for each point.<br><br>---<br><br>##  National Brands with Strong Nutrition Profiles<br><br>According to a recent data-driven analysis examining nearly 2 million products, the top national food brands with consistently high nutrition scores include:<br><br>- **Bob’s Red Mill**: Offers minimally processed products rich in whole grains and nutrients, earning an excellent nutrition score of 8.4. ([ultraprocessedfoodlist.com](https://www.ultraprocessedfoodlist.com/guides/brands-by-nutrition?utm_source=openai))  <br>- **Chobani**: Known for its Greek yogurt and functional foods, scores around 6.1, reflecting good nutritional quality; their nutritious product line includes high-protein yogurt and plant-based options. ([ultraprocessedfoodlist.com](https://www.ultraprocessedfoodlist.com/guides/brands-by-nutrition?utm_source=openai))  <br>- **Planters** and **Atkins** each score around 6.4, offering relatively healthy snack and meal options compared to many ultra-processed competitors. ([ultraprocessedfoodlist.com](https://www.ultraprocessedfoodlist.com/guides/brands-by-nutrition?utm_source=openai))<br><br>Additionally, certain product categories—especially legume- and pasta-based items—excel in nutritional value:<br><br>- **Explore Cuisine**, **Tolerant**, **Pines**, **San Giorgio**, **Ronzoni**, **De Cecco**, and others deliver whole-grain or legume-rich pasta with excellent nutrition, reflected in scores above 9.0. ([ultraprocessedfoodlist.com](https://www.ultraprocessedfoodlist.com/guides/brands-by-nutrition?utm_source=openai))<br><br>---<br><br>##  Health-Focused Store Brands<br><br>Store brands often feature prominently among the healthiest choices:<br><br>- **Trader Joe’s** (5.7), **Kirkland Signature** (5.5), **O Organics** (5.4), **365 Everyday Value** (5.4), and **Good & Gather** (5.2) are top-ranking in terms of nutrition quality among store-labels. ([ultraprocessedfoodlist.com](https://www.ultraprocessedfoodlist.com/guides/brands-by-nutrition?utm_source=openai))<br><br>These brands balance affordability with nutrient-dense options for shoppers frequently seeking healthy alternatives.<br><br>---<br><br>##  Brands Highlighted by Experts and Media for Healthfulness<br><br>Several brands are specifically praised by nutrition experts and publications for their clean ingredients, functional benefits, or more nutritious profiles:<br><br>- **Mary’s Gone Original**: Whole-grain, seed-packed crackers that provide good protein and fiber with clean ingredients. ([eatthis.com](https://www.eatthis.com/packaged-foods-shockingly-healthy/?utm_source=openai))  <br><br>- **Cocomo Coconut Peanut Butter**: An organic, low-sugar nut butter with minimal ingredients and no artificial additives. ([eatthis.com](https://www.eatthis.com/packaged-foods-shockingly-healthy/?utm_source=openai))  <br><br>- **GoVerden Avocado Cups**: Portable, non-GMO vegan cups delivering heart-healthy fats and fiber with only simple components like avocado, sea salt, and lime. ([eatthis.com](https://www.eatthis.com/packaged-foods-shockingly-healthy/?utm_source=openai))  <br><br>- **Trader Joe’s Steamed Lentils**: Convenient, high-fiber, plant-based protein source ideal for salads or soups. ([eatthis.com](https://www.eatthis.com/packaged-foods-shockingly-healthy/?utm_source=openai))  <br><br>- **Wild Planet Sardines in Extra Virgin Olive Oil**: Nutrient-dense proteins rich in omega‑3s, without unnecessary additives. ([eatthis.com](https://www.eatthis.com/packaged-foods-shockingly-healthy/?utm_source=openai))  <br><br>Media lists also spotlight standout items:<br><br>- **Banza chickpea pasta**, praised for its high protein and fiber content compared to standard pasta. ([prevention.com](https://www.prevention.com/food-nutrition/healthy-eating/g63084420/healthiest-processed-foods/?utm_source=openai))  <br><br>- **Dr. Praeger’s**, **Saffron Road**, and **Banza** frozen meals recognized for healthier frozen food alternatives. ([flavor365.com](https://flavor365.com/the-10-best-healthy-frozen-food-brands-to-buy-in-2025/?utm_source=openai))  <br><br>- **Seven Sundays Oat Protein Cereal**, **Ezekiel 4:9 Flake Cereal**, **Barilla Whole Grain Thin Spaghetti**, **Banza Penne**, plus snacks like **Siete Grain-Free Chips** and **Dave’s Killer Bread Organic Snack Bars**, acknowledged in a 2023 Health food awards for balancing taste, affordability, and nutrition. ([health.com](https://www.health.com/food-awards-2023-8349365?utm_source=openai))  <br><br>---<br><br>##  Brands with Clean, Purpose-Driven Nutrition<br><br>Emerging and mission-driven brands are also noted for their commitment to health, ethics, and transparency:<br><br>- **Vital Farms**: Known for pasture‑raised eggs rich in omega‑3s and free from antibiotics, with regenerative agriculture initiatives introduced in 2025. ([thefoodiediary.com](https://thefoodiediary.com/top-food-brands-committed-to-health-and-wellness/?utm_source=openai))  <br><br>- **Impossible Foods**: Their plant-based burgers and other meat alternatives are fortified (e.g. with B12 and iron), low in cholesterol, and position themselves as climate-friendly and nutritious. ([thefoodiediary.com](https://thefoodiediary.com/top-food-brands-committed-to-health-and-wellness/?utm_source=openai))  <br><br>- **KIND Snacks**: Focuses on recognizable, wholesome ingredients (nuts, seeds) with low sugar and no artificial additives. Their newer higher-protein lines offer 10–12 g of protein per bar. ([thefoodiediary.com](https://thefoodiediary.com/top-food-brands-committed-to-health-and-wellness/?utm_source=openai))  <br><br>---<br><br>##  Consumer Perceptions: Brands Seen as Healthier<br><br>A pilot consumer survey ranked the perceived healthfulness of various brands, where some of the highest-rated include:<br><br>- **Healthy Choice** and **Kashi®** (both with a rating of 7.4), followed by **Quaker®** (7.3) and **Nature Valley®** (7.2). These brands are regarded more favorably by consumers than big-name processed brands like Kraft or Nabisco. ([pmc.ncbi.nlm.nih.gov](https://pmc.ncbi.nlm.nih.gov/articles/PMC7431378/?utm_source=openai))<br><br>---<br><br>##  Summary Overview<br><br>Here’s a consolidated list of brands recognized for overall healthfulness:<br><br>• Bob’s Red Mill  <br>• Chobani  <br>• Planters  <br>• Explore Cuisine, San Giorgio, Ronzoni, etc. (high‑nutrition pastas)  <br>• Trader Joe’s, Kirkland Signature, O Organics, 365 Everyday Value (store brands)  <br>• Mary’s Gone Original, Cocomo Coconut Peanut Butter, GoVerden Avocado Cups, Wild Planet Sardines  <br>• Banza, Dr. Praeger’s, Saffron Road (nutritious frozen and grain‑based products)  <br>• Seven Sundays, Ezekiel cereal, Siete Chips, Dave’s Killer Bread  <br>• Vital Farms, Impossible Foods, KIND Snacks (clean‑ingredient, mission‑driven brands)  <br>• Healthy Choice, Kashi, Quaker, Nature Valley (positively perceived by consumers)  <br><br>---<br><br>### Final Thoughts<br><br>- Bulk nutrition metrics favor minimally processed, whole-grain, legume-based, and fortified frozen foods.<br>- Store-brand lines and mission-driven brands often combine ethics with healthier formulations.<br>- Packaged staples like innovative cereals, snack bars, and frozen meals continue improving in nutritional value.<br>- Always cross-reference with new labeling — especially after the FDA’s updated definition of “healthy” (effective February 2028), which will mandate inclusion of fruit/vegetable/whole grain/protein and limits on added sugar, sodium, and saturated fat. ([apnews.com](https://apnews.com/article/e7afbe9020976ce45c911cebde2f1c0b?utm_source=openai))<br><br>If you'd like tailored recommendations based on your dietary needs—whether gluten-free, plant-based, low added sugar, or high-protein—I'm happy to help dig deeper! | 365 everyday value – brand<br>atkins – brand<br>banza – brand<br>banza chickpea pasta – product<br>banza penne – product<br>barilla – brand<br>barilla whole grain thin spaghetti – product<br>bob’s red mill – brand<br>chobani – brand<br>cocomo coconut peanut butter – product<br>dave’s killer bread – brand<br>dave’s killer bread organic snack bars – product<br>de cecco – brand<br>dr. praeger’s – brand<br>explore cuisine – brand<br>ezekiel 4:9 flake cereal – product<br>ezekiel cereal – brand<br>good & gather – brand<br>goverden avocado cups – product<br>healthy choice – brand<br>impossible foods – brand<br>kashi – brand<br>kind snacks – brand<br>kirkland signature – brand<br>kraft – brand<br>mary’s gone original – brand<br>nabisco – brand<br>nature valley – brand<br>o organics – brand<br>pines – brand<br>planters – brand<br>quaker – brand<br>ronzoni – brand<br>saffron road – brand<br>san giorgio – brand<br>seven sundays – brand<br>seven sundays oat protein cereal – product<br>siete chips – brand<br>siete grain-free chips – product<br>tolerant – brand<br>trader joe’s – brand<br>trader joe’s steamed lentils – product<br>vital farms – brand<br>wild planet sardines – brand<br>wild planet sardines in extra virgin olive oil – product | 365 everyday value – brand<br>atkins – brand<br>banza – brand<br>banza chickpea pasta – product<br>banza penne – product<br>barilla whole grain thin spaghetti – product<br>bob’s red mill – brand<br>chobani – brand<br>cocomo coconut peanut butter – product<br>dave’s killer bread organic snack bar – product<br>de cecco – brand<br>dr. praeger’s – brand<br>explore cuisine – brand<br>ezekiel 4:9 flake cereal – product<br>good & gather – brand<br>goverden avocado cup – product<br>healthy choice – brand<br>impossible foods – brand<br>kashi – brand<br>kind snack – brand<br>kirkland signature – brand<br>mary’s gone original – brand<br>nature valley – brand<br>o organics – brand<br>pine – brand<br>planter – brand<br>quaker – brand<br>ronzoni – brand<br>saffron road – brand<br>san giorgio – brand<br>seven sundays oat protein cereal – product<br>siete grain-free chip – product<br>tolerant – brand<br>trader joe’s – brand<br>trader joe’s steamed lentil – product<br>vital farms – brand<br>wild planet sardine in extra virgin olive oil – product | 365 everyday value – brand<br>atkins – brand<br>avocado cups – product<br>banza – brand<br>banza chickpea pasta – product<br>banza penne – product<br>barilla – brand<br>barilla whole grain thin spaghetti – product<br>bob's red mill – brand<br>chickpea pasta – product<br>chobani – brand<br>cocomo coconut peanut butter – product<br>dave’s killer bread – brand<br>de cecco – brand<br>dr. praeger’s – brand<br>explore cuisine – brand<br>ezekiel – brand<br>flake cereal – product<br>good & gather – brand<br>goverden – brand<br>grain-free chips – product<br>healthy choice – brand<br>impossible foods – brand<br>kashi – brand<br>kind snacks – brand<br>kirkland signature – brand<br>mary’s gone original – brand<br>mary’s gone original crackers – product<br>nature valley – brand<br>o organics – brand<br>oat protein cereal – product<br>organic snack bar – product<br>penne – product<br>pines – brand<br>planters – brand<br>quaker – brand<br>ronzoni – brand<br>saffron road – brand<br>san giorgio – brand<br>sardines in extra virgin olive oil – product<br>seven sundays – brand<br>siete – brand<br>tolerant – brand<br>trader joe's – brand<br>trader joe's steamed lentils – product<br>vital farms – brand<br>wild planet – brand<br>wild planet sardines in extra virgin olive oil – product | 365 everyday value – brand<br>atkins – brand<br>banza – brand<br>banza chickpea pasta – product<br>banza penne – product<br>barilla whole grain thin spaghetti – product<br>bob’s red mill – brand<br>chobani – brand<br>cocomo coconut peanut butter – brand<br>dave’s killer bread organic snack bar – product<br>de cecco – brand<br>dr. praeger’s – brand<br>explore cuisine – brand<br>ezekiel 4:9 flake cereal – product<br>good & gather – brand<br>goverden avocado cups – brand<br>healthy choice – brand<br>impossible foods – brand<br>kashi – brand<br>kind snacks – brand<br>kirkland signature – brand<br>mary’s gone original – brand<br>nature valley – brand<br>o organics – brand<br>pines – brand<br>planters – brand<br>quaker – brand<br>ronzoni – brand<br>saffron road – brand<br>san giorgio – brand<br>seven sundays oat protein cereal – product<br>siete grain-free chips – product<br>tolerant – brand<br>trader joe’s – brand<br>trader joe’s steamed lentils – product<br>vital farms – brand<br>wild planet – brand<br>wild planet sardines in extra virgin olive oil – product | 365 everyday value – brand<br>atkins – brand<br>banza chickpea pasta – product<br>banza frozen meal – product<br>banza penne – product<br>barilla whole grain thin spaghetti – product<br>bob’s red mill – brand<br>chobani – brand<br>cocomo coconut peanut butter – product<br>dave’s killer bread organic snack bars – product<br>de cecco – brand<br>dr. praeger’s – brand<br>explore cuisine – brand<br>ezekiel 4:9 flake cereal – product<br>good & gather – brand<br>goverden avocado cups – product<br>healthy choice – brand<br>impossible foods – brand<br>kashi – brand<br>kind snacks – brand<br>kirkland signature – brand<br>mary’s gone original – brand<br>nature valley – brand<br>o organics – brand<br>pines – brand<br>planters – brand<br>quaker – brand<br>ronzoni – brand<br>saffron road – brand<br>san giorgio – brand<br>seven sundays oat protein cereal – product<br>siete grain-free chips – product<br>tolerant – brand<br>trader joe’s – brand<br>trader joe’s steamed lentils – product<br>vital farms – brand<br>wild planet sardines in extra virgin olive oil – product | 365 everyday value – brand<br>atkins – brand<br>banza – brand<br>barilla – brand<br>bob’s red mill – brand<br>chobani – brand<br>cocomo coconut peanut butter – brand<br>dave’s killer bread – brand<br>de cecco – brand<br>dr. praeger’s – brand<br>explore cuisine – brand<br>ezekiel 4:9 flake cereal – brand<br>good & gather – brand<br>goverden avocado cups – brand<br>healthy choice – brand<br>impossible foods – brand<br>kashi – brand<br>kind snacks – brand<br>kirkland signature – brand<br>mary’s gone original – brand<br>nature valley – brand<br>o organics – brand<br>pines – brand<br>planters – brand<br>quaker – brand<br>ronzoni – brand<br>saffron road – brand<br>san giorgio – brand<br>seven sundays – brand<br>siete – brand<br>tolerant – brand<br>trader joe’s – brand<br>vital farms – brand<br>wild planet sardines – brand |